Dans ce notebook nous entrainerons nos données sur les restes dees facultées et ensuite continuerons après

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns #for beatiful visualizations
%matplotlib inline 
import scipy.stats as scs #for statistics
import operator
from scipy.stats import chi2_contingency
import matplotlib.ticker as ticker
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import *

In [2]:
from sklearn.linear_model import Ridge,Lasso,ElasticNet,LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.svm import LinearSVR,SVR
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib

In [1]:
from codes.predictiveModelBuilding import PredictiveModelBuilding

In [6]:
from predictiveModelBuilding import final_job , convert_cat, build_final_dataset

ImportError: No module named predictiveModelBuilding

## Using the final function

In [5]:
dataset = pd.read_csv("../dataset/DatasetFinalV2.csv",index_col='ID')

In [50]:
school_list = list(dataset.SCHOOL_RIGHT.value_counts().index)

In [53]:
option_list = list(dataset.OPTION_RIGHT.value_counts().index)

In [24]:
import json,io

In [54]:
json.dumps(option_list)

'["pedagogie", "commmerciale et adm", "bio-chimie", "sociale", "latin philo", "math-physique", "commerciale informatique", "nutr", "mec gene", "construction", "elec indust", "elec", "coupe couture", "electronique g\\u00e9n\\u00e9rale", "inconnu", "mecanique machines outils", "v\\u00e9t\\u00e9rinaire", "agrecole", "secretariat", "machine outil", "hotesse d\'acceuil", "agronomie", "batiment", "hospitali\\u00e8re", "economie", "diet", "fdf", "industrielle", "relations publiques", "imprimerie"]'

In [22]:
results, predicted_results = final_job(dataframe=dataset)

          DIPPERC        CGPA
count  716.000000  716.000000
mean     0.560611    0.585911
std      0.049969    0.060723
min      0.500000    0.364000
25%      0.520000    0.550000
50%      0.550000    0.593667
75%      0.590000    0.624125
max      0.780000    0.781000
          DIPPERC        CGPA
count  180.000000  180.000000
mean     0.565104    0.583321
std      0.057813    0.060896
min      0.500000    0.421000
25%      0.520000    0.556250
50%      0.550000    0.589667
75%      0.600000    0.618938
max      0.780000    0.732000
[LibSVM]('Predictions:\t', array([ 0.59798621,  0.57821287,  0.65290232,  0.69264086,  0.49638616]))
('Labels:\t\t', [0.559000015259, 0.5360000038149999, 0.66199996948199991, 0.69700000762900005, 0.49249999999999999])
(ElasticNet(alpha=1e-05, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=20000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False), array([ 0.06479168,  

(Lasso(alpha=1e-05, copy_X=True, fit_intercept=False, max_iter=20000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False), array([ 0.05763573,  0.05800489,  0.0843771 ,  0.07064087,  0.05202378,
        0.05253309,  0.04128539,  0.05349263,  0.04487278,  0.05714517]))
('Predictions:\t', array([ 0.70965172,  0.52589337,  0.50557467,  0.71882489,  0.26180898]))
('Labels:\t\t', [0.60950000762899992, 0.59149999618500004, 0.5264999961850001, 0.71999999999999997, 0.625])
       ElasticNet     Lasso  LinearSVR     Ridge       SVR  RealValue  \
ID                                                                      
11897    0.584782  0.583048   0.547311  0.531813  0.583679     0.5720   
9993     0.527989  0.528960   0.537041  0.541294  0.574713     0.5270   
10585    0.636439  0.636927   0.642156  0.642995  0.603056     0.6040   
12075    0.653175  0.653705   0.644754  0.631774  0.583823     0.6440   
10674    0.613396 

(LinearSVR(C=1.0, dual=False, epsilon=0.0, fit_intercept=False,
     intercept_scaling=1.0, loss='squared_epsilon_insensitive',
     max_iter=20000, random_state=None, tol=0.0001, verbose=0), array([ 0.05083406,  0.06402964,  0.06881011,  0.05330369,  0.05465869,
        0.07021816,  0.0616327 ,  0.07094593,  0.06471843,  0.06265478]))
('Predictions:\t', array([ 0.63217337,  0.62797222,  0.58637013,  0.56070989,  0.55547788]))
('Labels:\t\t', [0.61649999618500007, 0.59949998855600006, 0.66600001017300003, 0.61399999618500001, 0.62400001525900006])
('Predictions:\t', array([ 0.60555608,  0.57725166,  0.62232088,  0.55331047,  0.60354096]))
('Labels:\t\t', [0.59999999999999998, 0.60900001525900005, 0.62966667175299995, 0.53924999237100002, 0.58949998855600005])
(Ridge(alpha=1, copy_X=True, fit_intercept=False, max_iter=20000,
   normalize=False, random_state=None, solver='cholesky', tol=0.001), array([ 0.04975508,  0.06440302,  0.06977786,  0.05373795,  0.0545419 ,
        0.06748977,  0

[LibSVM]('Predictions:\t', array([ 0.60543423,  0.62988871,  0.53243329,  0.52272051,  0.70562429]))
('Labels:\t\t', [0.57350000381499999, 0.68300003051800007, 0.50299999237100002, 0.49249999999999999, 0.76000000000000001])
(ElasticNet(alpha=1e-05, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=20000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False), array([ 0.07302507,  0.06527334,  0.06832415,  0.07041296,  0.07209081,
        0.07264586,  0.06942813,  0.07617049,  0.06419625,  0.06008306]))
('Predictions:\t', array([ 0.62152027,  0.57331892,  0.53060718,  0.61443687,  0.59856275]))
('Labels:\t\t', [0.61300001144399996, 0.613666687012, 0.41999999999999998, 0.53024999618500002, 0.62100000381499998])
('Predictions:\t', array([ 0.58127431,  0.5891881 ,  0.57599771,  0.56334776,  0.60104969]))
('Labels:\t\t', [0.57350000381499999, 0.68300003051800007, 0.50299999237100002, 0.49249999999999999, 0.76

(Lasso(alpha=1e-05, copy_X=True, fit_intercept=False, max_iter=20000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False), array([ 0.08310741,  0.07898499,  0.0626074 ,  0.15366031,  0.11042563,
        0.16835273,  0.05524483,  0.16616301,  0.09404036,  0.08652426]))
('Predictions:\t', array([ 0.54691023,  0.63974514,  0.64658365,  0.65830039,  0.6193395 ]))
('Labels:\t\t', [0.62099998474099993, 0.47999999999999998, 0.68733332316099993, 0.67699998855599997, 0.62400001525900006])
       ElasticNet     Lasso  LinearSVR     Ridge       SVR  RealValue  \
ID                                                                      
5023     0.634679  0.630684   0.595408  0.594512  0.600609   0.629000   
12230    0.480670  0.481150   0.490235  0.480818  0.580034   0.480000   
11659    0.666297  0.664446   0.610001  0.595072  0.603692   0.679000   
4638     0.644807  0.645240   0.639454  0.631110  0.600622   0.610000   
688

In [34]:
results, final_predicted=final_job(dataframe=dataset)

          DIPPERC        CGPA
count  716.000000  716.000000
mean     0.560611    0.581048
std      0.049969    0.069594
min      0.500000    0.364000
25%      0.520000    0.550000
50%      0.550000    0.593667
75%      0.590000    0.624125
max      0.780000    0.781000
          DIPPERC        CGPA
count  180.000000  180.000000
mean     0.565104    0.576927
std      0.057813    0.072549
min      0.500000    0.400000
25%      0.520000    0.556250
50%      0.550000    0.589667
75%      0.600000    0.618938
max      0.780000    0.732000
[LibSVM]('Predictions:\t', array([ 0.59816346,  0.57944339,  0.65096787,  0.69258772,  0.42413759]))
('Labels:\t\t', [0.559000015259, 0.54100000381499991, 0.66199996948199991, 0.69700000762900005, 0.41999999999999998])
(ElasticNet(alpha=1e-05, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=20000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False), array([ 0.0773987 , 

(Lasso(alpha=1e-05, copy_X=True, fit_intercept=False, max_iter=20000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False), array([ 0.06774149,  0.05756951,  0.08259653,  0.08371759,  0.07121494,
        0.05904762,  0.05570268,  0.07085067,  0.05647987,  0.07134523]))
('Predictions:\t', array([ 0.72034434,  0.4807772 ,  0.4921154 ,  0.72254536,  0.29952173]))
('Labels:\t\t', [0.60950000762899992, 0.59149999618500004, 0.5264999961850001, 0.71999999999999997, 0.625])
       ElasticNet     Lasso  LinearSVR     Ridge       SVR  RealValue  \
ID                                                                      
11897    0.576156  0.574638   0.541599  0.525965  0.548760     0.5720   
9993     0.527988  0.528954   0.533910  0.536457  0.531563     0.5270   
10585    0.636599  0.637144   0.641325  0.642037  0.602181     0.6040   
12075    0.655938  0.656493   0.643859  0.629510  0.563468     0.6440   
10674    0.610100 

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [23]:
final = build_final_dataset(results)

In [24]:
final.fillna(value=' ',inplace=True)

In [15]:
final.reset_index(inplace=True)

In [25]:
final.drop(labels=['index'],axis=1,inplace=True)

In [26]:
final = final[['Faculte','Dimensions','MODEL','RMSE Train','CVSCORE Mean', 'CVSCORE Std', 'RMSE Test' ,'STACK_RES' ]]

In [28]:
final.to_csv('../dataset/ResultDataset.csv')

In [31]:
import os
predictives_models = []
path = '../predictivesModels/Classes/'
for filename in os.listdir(path):
    print '-------------before load--------------'
    model = joblib.load(path+filename)
    predictives_models.append(model)
    print '-------------after load--------------'

-------------before load--------------
-------------after load--------------
-------------before load--------------
-------------after load--------------
-------------before load--------------
-------------after load--------------
-------------before load--------------
-------------after load--------------
-------------before load--------------
-------------after load--------------
-------------before load--------------
-------------after load--------------
-------------before load--------------
-------------after load--------------


In [33]:
x = predictives_models[0]

In [35]:
x.predict_new(new_student_data = {'DIPPERC':0.60, 'SCHOOL_RIGHT':'itfm/bukavu', 'OPTION_RIGHT':'elec indust'})

TypeError: unhashable type: 'list'

# final_predicted

In [14]:
droit=datasetCGPA.loc[datasetCGPA.FAC=='FD']

In [15]:
droit.shape

(896, 8)

In [16]:
droitModel=PredictiveModelBuilding(dataset=droit,encoderFunction=ConvertCat)

In [17]:
droitModel.scale(['CGPA','DIPPERC'])

In [18]:
describeTrain,describeTest=droitModel.split()

In [31]:
describeTest

,DIPPERC,CGPA
count,180.000000,180.000000
mean,0.565104,0.583321
std,0.057813,0.060896
min,0.500000,0.421000
25%,0.520000,0.556250
50%,0.550000,0.589667
75%,0.600000,0.618938
max,0.780000,0.732000


In [32]:
describeTrain

,DIPPERC,CGPA
count,716.000000,716.000000
mean,0.560611,0.585911
std,0.049969,0.060723
min,0.500000,0.364000
25%,0.520000,0.550000
50%,0.550000,0.593667
75%,0.590000,0.624125
max,0.780000,0.781000


In [33]:
droitModel.dataset_bin.shape

(896, 300)

In [34]:
droitModel.y_train.shape

(716,)

In [19]:
predictedValues=droitModel.train()

[LibSVM]

In [19]:
droitModel.predictiveModels.keys()

['ElasticNet', 'SVR', 'LinearSVR', 'Ridge', 'Lasso']

In [20]:
predictedValues.head(10)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
12035,0.597986,0.602794,0.627807,0.632074,0.577132,0.559000
10804,0.578213,0.576982,0.570929,0.566150,0.575894,0.536000
11613,0.652902,0.661854,0.718267,0.727048,0.582797,0.662000
8700,0.692641,0.689864,0.619541,0.577912,0.586796,0.697000
1295,0.496386,0.499684,0.509751,0.515676,0.577740,0.492500
12136,0.538667,0.538594,0.531304,0.530954,0.573926,0.492500
3672,0.631643,0.631455,0.631650,0.625346,0.576637,0.621000
9484,0.576682,0.573197,0.564425,0.555882,0.573926,0.580333
8752,0.550403,0.553935,0.546796,0.546298,0.576142,0.546750


In [22]:
RMSE={}
for reg in droitModel.predictive_models.keys():
    rmse=droitModel.evaluate(model=reg,sur='train') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/droitModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.59798621,  0.57821287,  0.65290232,  0.69264086,  0.49638616]))
('Labels:\t\t', [0.559000015259, 0.5360000038149999, 0.66199996948199991, 0.69700000762900005, 0.49249999999999999])
('Predictions:\t', array([ 0.57713174,  0.57589436,  0.58279729,  0.58679567,  0.5777401 ]))
('Labels:\t\t', [0.559000015259, 0.5360000038149999, 0.66199996948199991, 0.69700000762900005, 0.49249999999999999])
('Predictions:\t', array([ 0.62780656,  0.5709293 ,  0.71826659,  0.61954091,  0.50975122]))
('Labels:\t\t', [0.559000015259, 0.5360000038149999, 0.66199996948199991, 0.69700000762900005, 0.49249999999999999])
('Predictions:\t', array([ 0.63207441,  0.56614967,  0.72704776,  0.57791221,  0.51567642]))
('Labels:\t\t', [0.559000015259, 0.5360000038149999, 0.66199996948199991, 0.69700000762900005, 0.49249999999999999])
('Predictions:\t', array([ 0.60279409,  0.57698191,  0.66185373,  0.68986369,  0.49968415]))
('Labels:\t\t', [0.559000015259, 0.5360000038149999, 0.661999969481

In [23]:
RMSE

{'ElasticNet': [0.041674337005922282, 7.1190669106863771],
 'Lasso': [0.041814324526193075, 7.1429804410522513],
 'LinearSVR': [0.045374153163771393, 7.7510922931471304],
 'Ridge': [0.048216161339260953, 8.2365816330051995],
 'SVR': [0.058326056148139099, 9.9636161289391456]}

Let try cross validation

In [24]:
CVScore={}
for reg in droitModel.predictive_models.keys():
    scores,Sstd,Smean=droitModel.cross_evaluate(model=reg) #RMSE of each model
    CVScore[reg]=[Smean,Smean*100/droitModel.dataset_bin.CGPA.mean(),Sstd]

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [25]:
CVScore

{'ElasticNet': [0.060485519212146045,
  10.332508909204291,
  0.0049199034174401647],
 'Lasso': [0.07071401186145701, 12.079803018659723, 0.014304328552641651],
 'LinearSVR': [0.06607627496349297, 11.287556238919155, 0.0087704488440109359],
 'Ridge': [0.065843115386304135, 11.247726483963755, 0.0083699395861444609],
 'SVR': [0.059504513399358847, 10.164927454458846, 0.0038795957315208014]}

Appres les scrores sur la validation croisée essayons de combinner les methodes par une rgression lineaire

In [54]:
from sklearn.model_selection import GridSearchCV
param_grid ={'alpha':[1e-5,1e-4, 1e-3,1e-2,0, 1, 5, 10],'selection':['cyclic','random']}
grid_search = GridSearchCV(droitModel.predictiveModels['Lasso'], param_grid, cv=5,scoring='neg_mean_squared_error',verbose=5)
grid_search.fit(droitModel.X_train,droitModel.Y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] alpha=1e-05, selection=cyclic ...................................
[CV] ... alpha=1e-05, selection=cyclic, score=-0.005322, total=   0.3s
[CV] alpha=1e-05, selection=cyclic ...................................
[CV] ... alpha=1e-05, selection=cyclic, score=-0.004703, total=   0.1s
[CV] alpha=1e-05, selection=cyclic ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV] ... alpha=1e-05, selection=cyclic, score=-0.005221, total=   0.3s
[CV] alpha=1e-05, selection=cyclic ...................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s


[CV] ... alpha=1e-05, selection=cyclic, score=-0.005055, total=   0.2s
[CV] alpha=1e-05, selection=cyclic ...................................
[CV] ... alpha=1e-05, selection=cyclic, score=-0.004535, total=   0.1s
[CV] alpha=1e-05, selection=random ...................................


...

[CV] ... alpha=1e-05, selection=random, score=-0.005322, total=   0.4s
[CV] alpha=1e-05, selection=random ...................................
[CV] ... alpha=1e-05, selection=random, score=-0.004703, total=   0.3s
[CV] alpha=1e-05, selection=random ...................................
[CV] ... alpha=1e-05, selection=random, score=-0.005229, total=   0.2s
[CV] alpha=1e-05, selection=random ...................................
[CV] ... alpha=1e-05, selection=random, score=-0.005055, total=   0.1s
[CV] alpha=1e-05, selection=random ...................................
[CV] ... alpha=1e-05, selection=random, score=-0.004535, total=   0.2s
[CV] alpha=0.0001, selection=cyclic ..................................
[CV] .. alpha=0.0001, selection=cyclic, score=-0.004921, total=   0.0s
[CV] alpha=0.0001, selection=cyclic ..................................
[CV] .. alpha=0.0001, selection=cyclic, score=-0.004540, total=   0.1s
[CV] alpha=0.0001, selection=cyclic ..................................
[CV] .

/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:238: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:470: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  positive)
/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[CV] ....... alpha=0, selection=cyclic, score=-0.005376, total=   4.1s
[CV] alpha=0, selection=cyclic .......................................
[CV] ....... alpha=0, selection=cyclic, score=-0.004736, total=   5.0s
[CV] alpha=0, selection=cyclic .......................................
[CV] ....... alpha=0, selection=cyclic, score=-0.005348, total=   5.0s
[CV] alpha=0, selection=cyclic .......................................
[CV] ....... alpha=0, selection=cyclic, score=-0.005173, total=   4.8s
[CV] alpha=0, selection=cyclic .......................................
[CV] ....... alpha=0, selection=cyclic, score=-0.004637, total=   4.7s
[CV] alpha=0, selection=random .......................................
[CV] ....... alpha=0, selection=random, score=-0.005372, total=   3.8s
[CV] alpha=0, selection=random .......................................
[CV] ....... alpha=0, selection=random, score=-0.004732, total=   4.9s
[CV] alpha=0, selection=random .......................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:   51.7s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=10000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0, 1, 5, 10], 'selection': ['cyclic', 'random']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=5)

In [55]:
cvres = grid_search.cv_results_
for mean_score, params in zip(sorted(cvres["mean_test_score"],reverse=True), cvres["params"]):
    print(np.sqrt(-mean_score), params)

(0.0672890169473533, {'alpha': 1e-05, 'selection': 'cyclic'})
(0.067290284332038222, {'alpha': 1e-05, 'selection': 'random'})
(0.067831595702809055, {'alpha': 0.0001, 'selection': 'cyclic'})
(0.067831710365960446, {'alpha': 0.0001, 'selection': 'random'})
(0.06957552507642617, {'alpha': 0.001, 'selection': 'cyclic'})
(0.06957552507642617, {'alpha': 0.001, 'selection': 'random'})
(0.06957552507642617, {'alpha': 0.01, 'selection': 'cyclic'})
(0.06957552507642617, {'alpha': 0.01, 'selection': 'random'})
(0.06957552507642617, {'alpha': 0, 'selection': 'cyclic'})
(0.06957552507642617, {'alpha': 0, 'selection': 'random'})
(0.06957552507642617, {'alpha': 1, 'selection': 'cyclic'})
(0.06957552507642617, {'alpha': 1, 'selection': 'random'})
(0.070481873651514551, {'alpha': 5, 'selection': 'cyclic'})
(0.070493598520100439, {'alpha': 5, 'selection': 'random'})
(0.071009492129496574, {'alpha': 10, 'selection': 'cyclic'})
(0.071094610279614737, {'alpha': 10, 'selection': 'random'})


Nous venons de voir qu'avec un alpha de 1e-05 notre modele dispose d'un bon score

In [56]:
from sklearn.model_selection import GridSearchCV
param_grid ={'alpha':[1e-5,1e-4, 1e-3,1e-2,0, 1, 5, 10],'selection':['cyclic','random'],'l1_ratio':[1,0.5,1.0/3.0,0.25,0.2]}
grid_search = GridSearchCV(droitModel.predictiveModels['ElasticNet'], param_grid, cv=5,scoring='neg_mean_squared_error',verbose=5)
grid_search.fit(droitModel.X_train,droitModel.Y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV] alpha=1e-05, selection=cyclic, l1_ratio=1 .......................
[CV]  alpha=1e-05, selection=cyclic, l1_ratio=1, score=-0.005322, total=   0.4s
[CV] alpha=1e-05, selection=cyclic, l1_ratio=1 .......................
[CV]  alpha=1e-05, selection=cyclic, l1_ratio=1, score=-0.004703, total=   0.1s
[CV] alpha=1e-05, selection=cyclic, l1_ratio=1 .......................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s


[CV]  alpha=1e-05, selection=cyclic, l1_ratio=1, score=-0.005221, total=   0.3s
[CV] alpha=1e-05, selection=cyclic, l1_ratio=1 .......................
[CV]  alpha=1e-05, selection=cyclic, l1_ratio=1, score=-0.005055, total=   0.1s
[CV] alpha=1e-05, selection=cyclic, l1_ratio=1 .......................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s


[CV]  alpha=1e-05, selection=cyclic, l1_ratio=1, score=-0.004535, total=   0.2s
[CV] alpha=1e-05, selection=random, l1_ratio=1 .......................
[CV]  alpha=1e-05, selection=random, l1_ratio=1, score=-0.005322, total=   0.4s
[CV] alpha=1e-05, selection=random, l1_ratio=1 .......................
[CV]  alpha=1e-05, selection=random, l1_ratio=1, score=-0.004703, total=   0.2s
[CV] alpha=1e-05, selection=random, l1_ratio=1 .......................
[CV]  alpha=1e-05, selection=random, l1_ratio=1, score=-0.005173, total=   0.1s
[CV] alpha=1e-05, selection=random, l1_ratio=1 .......................
[CV]  alpha=1e-05, selection=random, l1_ratio=1, score=-0.005055, total=   0.2s
[CV] alpha=1e-05, selection=random, l1_ratio=1 .......................
[CV]  alpha=1e-05, selection=random, l1_ratio=1, score=-0.004530, total=   0.2s
[CV] alpha=1e-05, selection=cyclic, l1_ratio=0.5 .....................
[CV]  alpha=1e-05, selection=cyclic, l1_ratio=0.5, score=-0.005359, total=   0.3s
[CV] alpha=1

...

[CV]  alpha=0, selection=cyclic, l1_ratio=0.25, score=-0.004637, total=   6.1s
[CV] alpha=0, selection=random, l1_ratio=0.25 ........................
[CV]  alpha=0, selection=random, l1_ratio=0.25, score=-0.005372, total=   4.1s
[CV] alpha=0, selection=random, l1_ratio=0.25 ........................
[CV]  alpha=0, selection=random, l1_ratio=0.25, score=-0.004767, total=   5.0s
[CV] alpha=0, selection=random, l1_ratio=0.25 ........................
[CV]  alpha=0, selection=random, l1_ratio=0.25, score=-0.005285, total=   5.7s
[CV] alpha=0, selection=random, l1_ratio=0.25 ........................
[CV]  alpha=0, selection=random, l1_ratio=0.25, score=-0.005162, total=   5.6s
[CV] alpha=0, selection=random, l1_ratio=0.25 ........................
[CV]  alpha=0, selection=random, l1_ratio=0.25, score=-0.004676, total=   5.2s
[CV] alpha=0, selection=cyclic, l1_ratio=0.2 .........................
[CV]  alpha=0, selection=cyclic, l1_ratio=0.2, score=-0.005376, total=   4.0s
[CV] alpha=0, selectio

[CV]  alpha=1, selection=cyclic, l1_ratio=0.2, score=-0.005463, total=   0.0s
[CV] alpha=1, selection=cyclic, l1_ratio=0.2 .........................
[CV]  alpha=1, selection=cyclic, l1_ratio=0.2, score=-0.005285, total=   0.0s
[CV] alpha=1, selection=cyclic, l1_ratio=0.2 .........................
[CV]  alpha=1, selection=cyclic, l1_ratio=0.2, score=-0.003664, total=   0.0s
[CV] alpha=1, selection=cyclic, l1_ratio=0.2 .........................
[CV]  alpha=1, selection=cyclic, l1_ratio=0.2, score=-0.004268, total=   0.0s
[CV] alpha=1, selection=random, l1_ratio=0.2 .........................
[CV]  alpha=1, selection=random, l1_ratio=0.2, score=-0.005519, total=   0.0s
[CV] alpha=1, selection=random, l1_ratio=0.2 .........................
[CV]  alpha=1, selection=random, l1_ratio=0.2, score=-0.005463, total=   0.0s
[CV] alpha=1, selection=random, l1_ratio=0.2 .........................
[CV]  alpha=1, selection=random, l1_ratio=0.2, score=-0.005285, total=   0.0s
[CV] alpha=1, selection=rand

[CV]  alpha=10, selection=random, l1_ratio=1, score=-0.005519, total=   0.0s
[CV] alpha=10, selection=random, l1_ratio=1 ..........................
[CV]  alpha=10, selection=random, l1_ratio=1, score=-0.005463, total=   0.0s
[CV] alpha=10, selection=random, l1_ratio=1 ..........................
[CV]  alpha=10, selection=random, l1_ratio=1, score=-0.005285, total=   0.0s
[CV] alpha=10, selection=random, l1_ratio=1 ..........................
[CV]  alpha=10, selection=random, l1_ratio=1, score=-0.003664, total=   0.0s
[CV] alpha=10, selection=random, l1_ratio=1 ..........................
[CV]  alpha=10, selection=random, l1_ratio=1, score=-0.004268, total=   0.0s
[CV] alpha=10, selection=cyclic, l1_ratio=0.5 ........................
[CV]  alpha=10, selection=cyclic, l1_ratio=0.5, score=-0.005519, total=   0.0s
[CV] alpha=10, selection=cyclic, l1_ratio=0.5 ........................
[CV]  alpha=10, selection=cyclic, l1_ratio=0.5, score=-0.005463, total=   0.0s
[CV] alpha=10, selection=cyclic

[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  4.6min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=10000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0, 1, 5, 10], 'selection': ['cyclic', 'random'], 'l1_ratio': [1, 0.5, 0.3333333333333333, 0.25, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=5)

In [57]:
cvres = grid_search.cv_results_
for mean_score, params in zip(sorted(cvres["mean_test_score"],reverse=True), cvres["params"]):
    print(np.sqrt(-mean_score), params)

(0.06667432276156228, {'alpha': 1e-05, 'selection': 'cyclic', 'l1_ratio': 1})
(0.066674559842560432, {'alpha': 1e-05, 'selection': 'random', 'l1_ratio': 1})
(0.066684532520481191, {'alpha': 1e-05, 'selection': 'cyclic', 'l1_ratio': 0.5})
(0.066685195121775606, {'alpha': 1e-05, 'selection': 'random', 'l1_ratio': 0.5})
(0.066752183580841659, {'alpha': 1e-05, 'selection': 'cyclic', 'l1_ratio': 0.3333333333333333})
(0.066752199918334462, {'alpha': 1e-05, 'selection': 'random', 'l1_ratio': 0.3333333333333333})
(0.067288550808824205, {'alpha': 1e-05, 'selection': 'cyclic', 'l1_ratio': 0.25})
(0.0672890169473533, {'alpha': 1e-05, 'selection': 'random', 'l1_ratio': 0.25})
(0.067373190313279152, {'alpha': 1e-05, 'selection': 'cyclic', 'l1_ratio': 0.2})
(0.067373433821402373, {'alpha': 1e-05, 'selection': 'random', 'l1_ratio': 0.2})
(0.067827927937534477, {'alpha': 0.0001, 'selection': 'cyclic', 'l1_ratio': 1})
(0.067831595702809055, {'alpha': 0.0001, 'selection': 'random', 'l1_ratio': 1})
(0.06

Grace à ces recherches nous avons pu evaluer nos paramentres sans problèmes

effectuons une evaluation sur le test set

In [26]:
RMSE={}
for reg in droitModel.predictive_models.keys():
    rmse=droitModel.evaluate(model=reg,sur='test') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/droitModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.64314557,  0.61909942,  0.60709689,  0.53532255,  0.6076823 ]))
('Labels:\t\t', [0.64033332824699996, 0.56233334859200002, 0.59999998728400006, 0.45400001525900002, 0.61950000762899993])
('Predictions:\t', array([ 0.57516252,  0.57754433,  0.58016663,  0.57400875,  0.58125158]))
('Labels:\t\t', [0.64033332824699996, 0.56233334859200002, 0.59999998728400006, 0.45400001525900002, 0.61950000762899993])
('Predictions:\t', array([ 0.67625126,  0.65325366,  0.54826038,  0.52413322,  0.60496434]))
('Labels:\t\t', [0.64033332824699996, 0.56233334859200002, 0.59999998728400006, 0.45400001525900002, 0.61950000762899993])
('Predictions:\t', array([ 0.67720796,  0.65945286,  0.52486378,  0.52529194,  0.6041236 ]))
('Labels:\t\t', [0.64033332824699996, 0.56233334859200002, 0.59999998728400006, 0.45400001525900002, 0.61950000762899993])
('Predictions:\t', array([ 0.64714841,  0.62035796,  0.60051424,  0.5347112 ,  0.60725181]))
('Labels:\t\t', [0.64033332824699996, 0.562

In [46]:
RMSE

{'ElasticNet': [0.061822094768174435, 10.560830977368248],
 'Lasso': [0.061707333909352735, 10.541226820676723],
 'LinearSVR': [0.065069638027064822, 11.115596317776752],
 'Ridge': [0.064775817566256541, 11.065404097084061],
 'SVR': [0.059390568156997697, 10.14546262639432]}

Essayons now ls combinaison de plusieurs methodes

In [27]:
finalPredict,rmseFinal = droitModel.ensemble_methods(predictedValues)

/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [52]:
finalPredict.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue,finalPredict
ID,,,,,,,
12035,0.597986,0.602794,0.627807,0.632074,0.577132,0.5590,0.598471
10804,0.578213,0.576982,0.570929,0.566150,0.575894,0.5360,0.578899
11613,0.652902,0.661854,0.718267,0.727048,0.582797,0.6620,0.655170
8700,0.692641,0.689864,0.619541,0.577912,0.586796,0.6970,0.697109
1295,0.496386,0.499684,0.509751,0.515676,0.577740,0.4925,0.493361


In [53]:
rmseFinal

0.04164083956131856

In [28]:
rmseFinal*100/droitModel.dataset_bin.CGPA.mean()

7.1133446708955672

Nous venons de voir qu'on obtient  un score dans le 7% pres en combinant les differentes score par une regression lineaire

sur l'ensemble d'apprentissage gobale

essayons sur le test set enfin

In [30]:
predictionTest=droitModel.predict_test()

In [31]:
predictionTest.head(1)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
7748,0.643146,0.647148,0.676251,0.677208,0.575163,0.640333


In [34]:
finalPredictTes,rmseFinalTest=droitModel.ensemble_methods(predictionTest)

In [35]:
rmseFinalTest*100/droitModel.y_train.mean()

9.5817925283780045

Nous voici arriver à la fin de l'entrainement de nos modeles en faculté de droitm'

enregistrons la classe qui predit

In [36]:
droitModel.save_models('droit')

##### 3. Facuté de santé et devllopement Communautaire

In [52]:
deplist=[]
for departement,dataset in datasetCGPA.groupby('FAC'):
   deplist.append(departement)
departement=zip(deplist,['droit','medecine','psycologie','sante','economie','techonologie','theologie'])
departement

[('FD', 'droit'),
 ('FM', 'medecine'),
 ('FPSE', 'psycologie'),
 ('FSDC', 'sante'),
 ('FSEG', 'economie'),
 ('FSTA', 'techonologie'),
 ('FT', 'theologie')]

In [55]:
sante=datasetCGPA.loc[datasetCGPA.FAC=='FSDC']

In [56]:
sante.head(5)

,DIPPERC,SCHOOL_RIGHT,OPTION_RIGHT,FAC,CGPA,DistinctionRatio,EchecRatio,Pass1stSessionRatio
ID,,,,,,,,
7365,62.0,zanner,latin philo,FSDC,63.400000,G,G,A
11712,54.0,zanner,latin philo,FSDC,60.000000,G,G,A
11862,54.0,mululusake,sociale,FSDC,61.900002,G,G,G
7003,50.0,ibanda,bio-chimie,FSDC,62.700001,G,G,G
7354,62.0,ibanda,commmerciale et adm,FSDC,49.250000,G,A,G


In [58]:
santeModel=PredictiveModelBuilding(dataset=sante,encoderFunction=ConvertCat)

In [59]:
santeModel.scale(['DIPPERC','CGPA'])

In [60]:
santeModel.dataset_bin.shape

(758, 257)

In [61]:
trainDes,tesDes=santeModel.split()

In [62]:
trainDes

,DIPPERC,CGPA
count,606.000000,606.000000
mean,0.552699,0.594791
std,0.048637,0.052894
min,0.500000,0.470000
25%,0.520000,0.573000
50%,0.540000,0.606000
75%,0.580000,0.629375
max,0.770000,0.724333


In [63]:
tesDes

,DIPPERC,CGPA
count,152.000000,152.000000
mean,0.555658,0.596356
std,0.051374,0.055168
min,0.500000,0.470000
25%,0.510000,0.571167
50%,0.550000,0.603833
75%,0.582500,0.634438
max,0.720000,0.708000


In [64]:
predictedValues=santeModel.train()

[LibSVM]

In [65]:
predictedValues.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
11728,0.603059,0.606095,0.603593,0.605556,0.577037,0.600000
4735,0.587404,0.586249,0.578260,0.577252,0.574934,0.609000
8748,0.607307,0.608028,0.621549,0.622321,0.578093,0.629667
4863,0.549911,0.550224,0.550819,0.553310,0.573313,0.539250
9186,0.601114,0.601241,0.602318,0.603541,0.573409,0.589500


In [68]:
RMSE={}
for reg in santeModel.predictive_models.keys():
    rmse=santeModel.evaluate(model=reg,sur='train') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/santeModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.60305942,  0.5874041 ,  0.60730708,  0.54991122,  0.60111387]))
('Labels:\t\t', [0.59999999999999998, 0.60900001525900005, 0.62966667175299995, 0.53924999237100002, 0.58949998855600005])
('Predictions:\t', array([ 0.57703736,  0.57493412,  0.578093  ,  0.57331305,  0.57340912]))
('Labels:\t\t', [0.59999999999999998, 0.60900001525900005, 0.62966667175299995, 0.53924999237100002, 0.58949998855600005])
('Predictions:\t', array([ 0.60359329,  0.57826044,  0.62154868,  0.55081946,  0.60231838]))
('Labels:\t\t', [0.59999999999999998, 0.60900001525900005, 0.62966667175299995, 0.53924999237100002, 0.58949998855600005])
('Predictions:\t', array([ 0.60555608,  0.57725166,  0.62232088,  0.55331047,  0.60354096]))
('Labels:\t\t', [0.59999999999999998, 0.60900001525900005, 0.62966667175299995, 0.53924999237100002, 0.58949998855600005])
('Predictions:\t', array([ 0.6060951 ,  0.58624896,  0.60802787,  0.55022353,  0.60124111]))
('Labels:\t\t', [0.59999999999999998, 0.609

In [69]:
RMSE

{'ElasticNet': [0.037874135533235195, 6.3642790910605864],
 'Lasso': [0.037981740957185831, 6.382360848969828],
 'LinearSVR': [0.043415709648957253, 7.2954719428500105],
 'Ridge': [0.046135012794505735, 7.7524171353358655],
 'SVR': [0.054615443951248024, 9.1774484907491782]}

on remarque aisement que les valeurs predites par differents regressons disposent d'une bonne exacitude

In [70]:
CVScore={}
for reg in santeModel.predictive_models.keys():
    scores,Sstd,Smean=santeModel.cross_evaluate(model=reg) #RMSE of each model
    CVScore[reg]=[Smean,Smean*100/santeModel.dataset_bin.CGPA.mean(),Sstd]

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [71]:
CVScore

{'ElasticNet': [0.053704993627275689,
  9.0244585972842355,
  0.004550254281595255],
 'Lasso': [0.06881415594533849, 11.56336607254282, 0.020902846770690368],
 'LinearSVR': [0.062180618561731016,
  10.448682326607152,
  0.0067815266150900964],
 'Ridge': [0.06167217553838468, 10.363244777817711, 0.0068876688637782221],
 'SVR': [0.055192478399835564, 9.2744119784767012, 0.003674694507116957]}

Aussi on peut remarquer les methodes lineaires disposent des meilleurs resulats

In [72]:
RMSE={}
for reg in santeModel.predictive_models.keys():
    rmse=santeModel.evaluate(model=reg,sur='test') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/santeModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.62440176,  0.60963392,  0.59897425,  0.58492151,  0.58002665]))
('Labels:\t\t', [0.61649999618500007, 0.59949998855600006, 0.66600001017300003, 0.61399999618500001, 0.62400001525900006])
('Predictions:\t', array([ 0.58187287,  0.57388956,  0.57350519,  0.57770715,  0.57459302]))
('Labels:\t\t', [0.61649999618500007, 0.59949998855600006, 0.66600001017300003, 0.61399999618500001, 0.62400001525900006])
('Predictions:\t', array([ 0.63217337,  0.62797222,  0.58637013,  0.56070989,  0.55547788]))
('Labels:\t\t', [0.61649999618500007, 0.59949998855600006, 0.66600001017300003, 0.61399999618500001, 0.62400001525900006])
('Predictions:\t', array([ 0.63076391,  0.62816821,  0.58529655,  0.55828218,  0.55291822]))
('Labels:\t\t', [0.61649999618500007, 0.59949998855600006, 0.66600001017300003, 0.61399999618500001, 0.62400001525900006])
('Predictions:\t', array([ 0.62759702,  0.61096817,  0.59739288,  0.58180375,  0.57803885]))
('Labels:\t\t', [0.61649999618500007, 0.599

In [73]:
RMSE

{'ElasticNet': [0.055455558954115379, 9.3186194051887501],
 'Lasso': [0.055006064386396075, 9.2430874138018844],
 'LinearSVR': [0.058031789289556324, 9.7515229850794416],
 'Ridge': [0.057961238859838594, 9.7396678597175992],
 'SVR': [0.057227150332930847, 9.6163133805457459]}

In [167]:
predictedValues.columns

Index([u'ElasticNet', u'Lasso', u'LinearSVR', u'Ridge', u'SVR', u'RealValue'], dtype='object')

In [75]:
finalPred,finalRMSE=santeModel.ensemble_methods(predictedValues)

In [76]:
finalPred.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue,finalPredict
ID,,,,,,,
11728,0.603059,0.606095,0.603593,0.605556,0.577037,0.600000,0.601301
4735,0.587404,0.586249,0.578260,0.577252,0.574934,0.609000,0.587466
8748,0.607307,0.608028,0.621549,0.622321,0.578093,0.629667,0.607796
4863,0.549911,0.550224,0.550819,0.553310,0.573313,0.539250,0.548800
9186,0.601114,0.601241,0.602318,0.603541,0.573409,0.589500,0.601170


In [77]:
finalRMSE*100/santeModel.dataset_bin.CGPA.mean()

6.36043795366355

On remarque que notre ensemble methode sans modele avec SVM nous donne un score de 8% en valdiation croisé voyons combien il nous donnerasavec l'ensemble d'evaluation

In [78]:
predictionTest=santeModel.predict_test()

In [79]:
predictionTest.head(1)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
11131,0.624402,0.627597,0.632173,0.630764,0.581873,0.6165


In [81]:
finalPredictTes,rmseFinalTest=santeModel.ensemble_methods(predictionTest)

In [83]:
rmseFinalTest*100/santeModel.y_train.mean()

8.5246977372321755

Nous avons un resulat de 9.9 % sur notre ensemble d'evaluation wouhhhhh........

In [86]:
santeModel.save_models('sante')

##### 3. Facuté de Psycologie

In [8]:
psyco=datasetCGPA.loc[datasetCGPA.FAC=='FPSE']

In [88]:
psyco.head(5)

,DIPPERC,SCHOOL_RIGHT,OPTION_RIGHT,FAC,CGPA,DistinctionRatio,EchecRatio,Pass1stSessionRatio
ID,,,,,,,,
11347,55.0,mululusake,pedagogie,FPSE,47.250000,G,A,G
7026,60.0,butembo,pedagogie,FPSE,53.325001,G,D,G
8487,57.0,butembo,pedagogie,FPSE,59.600000,G,G,F
10434,59.0,butembo,pedagogie,FPSE,59.700001,G,G,A
7487,52.0,chemchem,coupe couture,FPSE,56.025000,G,D,G


In [11]:
psyco.EchecRatio.value_counts()

G    180
A     30
D     12
E      5
Name: EchecRatio, dtype: int64

In [10]:
psyco.EchecRatio.loc[psyco.EchecRatio.isin(['E','F','C'])]='E'

/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [91]:
psycologieModel=PredictiveModelBuilding(dataset=psyco,encoderFunction=ConvertCat)

In [92]:
psycologieModel.scale(['DIPPERC','CGPA'])

In [93]:
trainDes,tesDes=psycologieModel.split()

In [219]:
trainDes

,DIPPERC,CGPA
count,181.000000,181.000000
mean,0.561264,0.597438
std,0.056302,0.079608
min,0.500000,0.400000
25%,0.520000,0.559500
50%,0.550000,0.609000
75%,0.590000,0.652667
max,0.770000,0.781750


In [220]:
tesDes

,DIPPERC,CGPA
count,46.000000,46.000000
mean,0.565000,0.588315
std,0.058224,0.066295
min,0.500000,0.430000
25%,0.510000,0.564500
50%,0.560000,0.601000
75%,0.600000,0.627000
max,0.750000,0.714000


In [94]:
predictedValues=psycologieModel.train()

[LibSVM]

In [95]:
predictedValues.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
10912,0.668789,0.669454,0.710007,0.712416,0.593556,0.6790
10785,0.569246,0.568941,0.551987,0.552366,0.575058,0.4750
5105,0.644995,0.645676,0.671051,0.672559,0.611144,0.7170
9246,0.618193,0.618140,0.634135,0.635935,0.591576,0.6230
8578,0.688544,0.687722,0.673014,0.665549,0.593060,0.6895


In [97]:
RMSE={}
for reg in psycologieModel.predictive_models.keys():
    rmse=psycologieModel.evaluate(model=reg,sur='train') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/psycologieModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.66878864,  0.56924641,  0.64499526,  0.61819297,  0.6885437 ]))
('Labels:\t\t', [0.678999977112, 0.47499999999999998, 0.71699996948199995, 0.62300000508599995, 0.68950000762899999])
('Predictions:\t', array([ 0.59355553,  0.57505814,  0.61114426,  0.59157579,  0.59306046]))
('Labels:\t\t', [0.678999977112, 0.47499999999999998, 0.71699996948199995, 0.62300000508599995, 0.68950000762899999])
('Predictions:\t', array([ 0.7100065 ,  0.55198726,  0.67105078,  0.63413476,  0.67301426]))
('Labels:\t\t', [0.678999977112, 0.47499999999999998, 0.71699996948199995, 0.62300000508599995, 0.68950000762899999])
('Predictions:\t', array([ 0.71241632,  0.5523663 ,  0.6725592 ,  0.63593534,  0.6655495 ]))
('Labels:\t\t', [0.678999977112, 0.47499999999999998, 0.71699996948199995, 0.62300000508599995, 0.68950000762899999])
('Predictions:\t', array([ 0.6694542 ,  0.56894093,  0.64567616,  0.61814013,  0.6877223 ]))
('Labels:\t\t', [0.678999977112, 0.47499999999999998, 0.7169999

In [86]:
RMSE

{'ElasticNet': [0.048439757351696131, 8.1330817865213696],
 'Lasso': [0.048506383027202178, 8.1442683014338755],
 'LinearSVR': [0.053311148653203064, 8.9509930650946377],
 'Ridge': [0.051086773887118013, 8.5775184053209639],
 'SVR': [0.069900647273839767, 11.73638581798169]}

on remarque aisement que les valeurs predites par differents regressons disposent d'une bonne exacitude

In [99]:
CVScore={}
for reg in psycologieModel.predictive_models.keys():
    scores,Sstd,Smean=psycologieModel.cross_evaluate(model=reg) #RMSE of each model
    CVScore[reg]=[Smean,Smean*100/psycologieModel.dataset_bin.CGPA.mean(),Sstd]

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [100]:
CVScore

{'ElasticNet': [0.069874633712249723,
  11.619733808233377,
  0.0070367630298314061],
 'Lasso': [0.076385620836863385, 12.702472038080904, 0.022050437156973714],
 'LinearSVR': [0.079337555379762351, 13.193361103046431, 0.013457184423710529],
 'Ridge': [0.081017273326983275, 13.472688154691467, 0.015662432073904727],
 'SVR': [0.067369766079210441, 11.203189297379312, 0.0078772802827785631]}

Aussi on peut remarquer les methodes lineaires disposent des meilleurs resulats

In [103]:
RMSE={}
for reg in psycologieModel.predictive_models.keys():
    rmse=psycologieModel.evaluate(model=reg,sur='test') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/psycologieModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.61414949,  0.56830139,  0.59347116,  0.62282371,  0.60113815]))
('Labels:\t\t', [0.47499999999999998, 0.582333335876, 0.62266666412399996, 0.59799999237099999, 0.495])
('Predictions:\t', array([ 0.5917407 ,  0.58376372,  0.59273045,  0.59240048,  0.59075151]))
('Labels:\t\t', [0.47499999999999998, 0.582333335876, 0.62266666412399996, 0.59799999237099999, 0.495])
('Predictions:\t', array([ 0.59370228,  0.52850499,  0.60882337,  0.70384729,  0.55895761]))
('Labels:\t\t', [0.47499999999999998, 0.582333335876, 0.62266666412399996, 0.59799999237099999, 0.495])
('Predictions:\t', array([ 0.59429772,  0.51920091,  0.61410726,  0.6934569 ,  0.55882177]))
('Labels:\t\t', [0.47499999999999998, 0.582333335876, 0.62266666412399996, 0.59799999237099999, 0.495])
('Predictions:\t', array([ 0.60726947,  0.56091044,  0.59407969,  0.61644388,  0.59350787]))
('Labels:\t\t', [0.47499999999999998, 0.582333335876, 0.62266666412399996, 0.59799999237099999, 0.495])


In [104]:
RMSE

{'ElasticNet': [0.056591276957597442, 9.410791000683469],
 'Lasso': [0.054807654256222145, 9.1141852096656386],
 'LinearSVR': [0.058113903398322425, 9.663994674043126],
 'Ridge': [0.059327508887681901, 9.8658100108167019],
 'SVR': [0.053827577240071728, 8.9512042617277459]}

In [167]:
predictedValues.columns

Index([u'ElasticNet', u'Lasso', u'LinearSVR', u'Ridge', u'SVR', u'RealValue'], dtype='object')

In [107]:
finalPred,finalRMSE=psycologieModel.ensemble_methods(predictedValues)

In [111]:
finalPred.loc[operator.__and__(finalPred.RealValue<0.50,finalPred.finalPredict<0.50)]

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue,finalPredict
ID,,,,,,,
10789,0.496000,0.496835,0.516431,0.526911,0.590752,0.4950,0.494853
11347,0.473528,0.474337,0.509151,0.527486,0.574317,0.4725,0.472383
8602,0.473523,0.474336,0.503367,0.518617,0.573823,0.4725,0.472428
10482,0.470999,0.471837,0.495199,0.503469,0.569933,0.4700,0.470105
10440,0.498504,0.499337,0.523880,0.537030,0.591246,0.4975,0.497330
9329,0.473519,0.474335,0.499512,0.512705,0.573493,0.4725,0.472458
10651,0.475051,0.471409,0.431736,0.416826,0.571248,0.4750,0.475029
8928,0.471048,0.471842,0.524835,0.552843,0.575802,0.4700,0.469725
5192,0.483514,0.484336,0.510030,0.523617,0.582563,0.4825,0.482369


In [113]:
finalRMSE*100/psycologieModel.dataset_bin.CGPA.mean()

6.4212727357281185

On remarque que notre ensemble methode sans modele avec SVM nous donne un score de 6% en valdiation croisé voyons combien il nous donnerasavec l'ensemble d'evaluation

In [115]:
predictionTest=psycologieModel.predict_test()

In [234]:
predictionTest.head(1)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
9149,0.546273,0.546273,0.541992,0.567743,0.576357,0.47


In [118]:
finalPredictTes,rmseFinalTest=psycologieModel.ensemble_methods(predictionTest)

In [120]:
rmseFinalTest*100/psycologieModel.y_train.mean()

7.9230724733661466

Nous avons un resulat de 10.5 % sur notre ensemble d'evaluation wouhhhhh........

 10.56

Efin atterisson avec la afaculté de theologie

In [125]:
psycologieModel.save_models('psycologie')

##### 3. Facuté de Theologie

In [12]:
teologie=datasetCGPA.loc[datasetCGPA.FAC=='FT']

In [122]:
teologie.shape

(140, 8)

In [123]:
teologie.head(5)

,DIPPERC,SCHOOL_RIGHT,OPTION_RIGHT,FAC,CGPA,DistinctionRatio,EchecRatio,Pass1stSessionRatio
ID,,,,,,,,
8692,52.0,mululusake,pedagogie,FT,59.933333,G,G,G
3651,58.0,ibanda,commmerciale et adm,FT,63.400002,G,G,G
11729,60.0,butembo,sociale,FT,64.400002,G,G,G
7512,52.0,ndosho,pedagogie,FT,66.875000,F,G,B
10686,52.0,ndosho,pedagogie,FT,57.100000,G,G,G


In [13]:
teologie.EchecRatio.value_counts()

G    119
A     14
E      4
D      2
F      1
Name: EchecRatio, dtype: int64

In [14]:
teologie.EchecRatio.loc[teologie.EchecRatio.isin(['E','D','F'])]='E'

/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [127]:
teologieModel=PredictiveModelBuilding(dataset=teologie,encoderFunction=ConvertCat)

In [128]:
teologieModel.scale(['DIPPERC','CGPA'])

In [129]:
trainDes,tesDes=teologieModel.split()

In [130]:
trainDes

,DIPPERC,CGPA
count,112.000000,112.000000
mean,0.538036,0.621912
std,0.039475,0.055595
min,0.500000,0.480000
25%,0.510000,0.594875
50%,0.520000,0.626000
75%,0.560000,0.656500
max,0.690000,0.732500


In [131]:
tesDes

,DIPPERC,CGPA
count,28.000000,28.000000
mean,0.538571,0.623810
std,0.043777,0.061440
min,0.500000,0.480000
25%,0.510000,0.593917
50%,0.520000,0.622500
75%,0.557500,0.669062
max,0.650000,0.734000


In [132]:
predictedValues=teologieModel.train()

[LibSVM]

In [133]:
predictedValues.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
5023,0.634679,0.630684,0.595408,0.594512,0.600609,0.629000
12230,0.480670,0.481150,0.490235,0.480818,0.580034,0.480000
11659,0.666297,0.664446,0.610001,0.595072,0.603692,0.679000
4638,0.644807,0.645240,0.639454,0.631110,0.600622,0.610000
6889,0.649849,0.658032,0.729154,0.726699,0.600845,0.689333


In [134]:
RMSE={}
for reg in teologieModel.predictive_models.keys():
    rmse = teologieModel.evaluate(model=reg,sur='train') #RMSE of each model
    RMSE[reg] = [rmse,rmse*100/teologieModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.63467891,  0.48066976,  0.66629706,  0.64480675,  0.64984864]))
('Labels:\t\t', [0.62900001525900007, 0.47999999999999998, 0.679000015259, 0.60999999999999999, 0.689333318074])
('Predictions:\t', array([ 0.6006087 ,  0.58003424,  0.60369197,  0.60062175,  0.60084506]))
('Labels:\t\t', [0.62900001525900007, 0.47999999999999998, 0.679000015259, 0.60999999999999999, 0.689333318074])
('Predictions:\t', array([ 0.59540829,  0.49023515,  0.61000084,  0.63945441,  0.72915369]))
('Labels:\t\t', [0.62900001525900007, 0.47999999999999998, 0.679000015259, 0.60999999999999999, 0.689333318074])
('Predictions:\t', array([ 0.5945123 ,  0.4808182 ,  0.59507176,  0.63111036,  0.72669892]))
('Labels:\t\t', [0.62900001525900007, 0.47999999999999998, 0.679000015259, 0.60999999999999999, 0.689333318074])
('Predictions:\t', array([ 0.63068435,  0.48115024,  0.66444634,  0.64523955,  0.65803173]))
('Labels:\t\t', [0.62900001525900007, 0.47999999999999998, 0.679000015259, 0.609999

In [135]:
RMSE

{'ElasticNet': [0.026076453609001484, 4.1903909230346912],
 'Lasso': [0.026127525931662329, 4.1985980588862644],
 'LinearSVR': [0.036800254033842718, 5.9136665123647623],
 'Ridge': [0.043905917120430356, 7.0555206366503258],
 'SVR': [0.055759250534385449, 8.9603080548441607]}

on remarque aisement que les valeurs predites par differents regressons disposent d'une bonne exacitude

In [136]:
CVScore={}
for reg in teologieModel.predictive_models.keys():
    scores,Sstd,Smean=teologieModel.cross_evaluate(model=reg) #RMSE of each model
    CVScore[reg]=[Smean,Smean*100/teologieModel.dataset_bin.CGPA.mean(),Sstd]

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [137]:
CVScore

{'ElasticNet': [0.073174307258027455,
  11.758844110134008,
  0.017187037554542334],
 'Lasso': [0.10591109298450707, 17.019526096054744, 0.04004208626083245],
 'LinearSVR': [0.080414973417988547, 12.922392735586351, 0.011395708866735618],
 'Ridge': [0.083404434880269634, 13.402788281857273, 0.011573412689346623],
 'SVR': [0.05848604815285182, 9.3984944800653327, 0.0092174990702130893]}

Aussi on peut remarquer les methodes lineaires disposent des meilleurs resulats

In [140]:
RMSE={}
for reg in teologieModel.predictive_models.keys():
    rmse=teologieModel.evaluate(model=reg,sur='test') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/teologieModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.5661005 ,  0.64028105,  0.64465214,  0.65583357,  0.62200086]))
('Labels:\t\t', [0.62099998474099993, 0.47999999999999998, 0.68733332316099993, 0.67699998855599997, 0.62400001525900006])
('Predictions:\t', array([ 0.60607815,  0.6006087 ,  0.60607815,  0.60072647,  0.6006087 ]))
('Labels:\t\t', [0.62099998474099993, 0.47999999999999998, 0.68733332316099993, 0.67699998855599997, 0.62400001525900006])
('Predictions:\t', array([ 0.44658671,  0.6278354 ,  0.57803651,  0.67594989,  0.58296778]))
('Labels:\t\t', [0.62099998474099993, 0.47999999999999998, 0.68733332316099993, 0.67699998855599997, 0.62400001525900006])
('Predictions:\t', array([ 0.43235251,  0.62219864,  0.55968019,  0.67104763,  0.58279997]))
('Labels:\t\t', [0.62099998474099993, 0.47999999999999998, 0.68733332316099993, 0.67699998855599997, 0.62400001525900006])
('Predictions:\t', array([ 0.54691023,  0.63974514,  0.64658365,  0.65830039,  0.6193395 ]))
('Labels:\t\t', [0.62099998474099993, 0.479

In [141]:
RMSE

{'ElasticNet': [0.069558152598374334, 11.177741254304362],
 'Lasso': [0.070528352534141192, 11.333648843017379],
 'LinearSVR': [0.086404831650951947, 13.884941092277385],
 'Ridge': [0.092437969866293962, 14.854444384176556],
 'SVR': [0.061988545836619717, 9.9613330746871345]}

In [167]:
predictedValues.columns

Index([u'ElasticNet', u'Lasso', u'LinearSVR', u'Ridge', u'SVR', u'RealValue'], dtype='object')

In [143]:
finalPred,finalRMSE=teologieModel.ensemble_methods(predictedValues)

In [144]:
finalPred.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue,finalPredict
ID,,,,,,,
5023,0.634679,0.630684,0.595408,0.594512,0.600609,0.629000,0.634600
12230,0.480670,0.481150,0.490235,0.480818,0.580034,0.480000,0.479630
11659,0.666297,0.664446,0.610001,0.595072,0.603692,0.679000,0.666771
4638,0.644807,0.645240,0.639454,0.631110,0.600622,0.610000,0.645083
6889,0.649849,0.658032,0.729154,0.726699,0.600845,0.689333,0.650563


In [145]:
finalRMSE*100/teologieModel.dataset_bin.CGPA.mean()

4.1898057159924855

On remarque que notre ensemble methode sans modele avec SVM nous donne un score de R% en valdiation croisé voyons combien il nous donnerasavec l'ensemble d'evaluation

In [146]:
predictionTest=teologieModel.predict_test()

In [147]:
predictionTest.head(1)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
10898,0.5661,0.54691,0.446587,0.432353,0.606078,0.621


In [150]:
finalPredictTes,rmseFinalTest=teologieModel.ensemble_methods(predictionTest)

In [151]:
rmseFinalTest*100/teologieModel.y_test.mean()

8.4857102081516267

Nous avons un resulat de 10.11 % sur notre ensemble d'evaluation wouhhhhh........

 10.56

In [153]:
teologieModel.save_models('theologie')

Passons encore à la Faculté de De Medecine

##### Faculté de Medecine

In [154]:
medecine=datasetCGPA.loc[datasetCGPA.FAC=='FM']

In [155]:
medecine.shape

(242, 8)

In [8]:
medecine.head(5)

,DIPPERC,SCHOOL_RIGHT,OPTION_RIGHT,FAC,CGPA,DistinctionRatio,EchecRatio,Pass1stSessionRatio
ID,,,,,,,,
10384,59.000000,zanner,latin philo,FM,59.150000,G,G,G
11610,52.000000,zanner,latin philo,FM,44.000000,G,A,G
10159,56.876522,iti gombe,vétérinaire,FM,58.049999,G,G,G
11776,51.000000,masisi,latin philo,FM,42.000000,G,A,G
10411,57.000000,ndosho,pedagogie,FM,49.000000,G,A,G


In [156]:
medecine.EchecRatio.value_counts()

G    172
A     49
D     21
Name: EchecRatio, dtype: int64

In [157]:
medecineModel=PredictiveModelBuilding(dataset=medecine,encoderFunction=ConvertCat)

In [158]:
medecineModel.scale(['DIPPERC','CGPA'])

In [159]:
trainDes,tesDes=medecineModel.split()

In [266]:
trainDes

,DIPPERC,CGPA
count,193.000000,193.000000
mean,0.594774,0.577718
std,0.072775,0.072102
min,0.500000,0.400000
25%,0.530000,0.547000
50%,0.580000,0.592500
75%,0.630000,0.622000
max,0.830000,0.740000


In [21]:
tesDes

,DIPPERC,CGPA
count,49.000000,49.000000
mean,0.592653,0.572949
std,0.075105,0.059417
min,0.500000,0.400000
25%,0.540000,0.551000
50%,0.580000,0.584500
75%,0.620000,0.610000
max,0.840000,0.698500


In [160]:
predictedValues=medecineModel.train()

[LibSVM]

In [161]:
predictedValues.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
11897,0.584782,0.583048,0.547311,0.531813,0.583679,0.5720
9993,0.527989,0.528960,0.537041,0.541294,0.574713,0.5270
10585,0.636439,0.636927,0.642156,0.642995,0.603056,0.6040
12075,0.653175,0.653705,0.644754,0.631774,0.583823,0.6440
10674,0.613396,0.612685,0.608467,0.609018,0.602563,0.5445


In [163]:
RMSE={}
for reg in medecineModel.predictive_models.keys():
    rmse=medecineModel.evaluate(model=reg,sur='train') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/medecineModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.58478237,  0.52798923,  0.63643927,  0.65317455,  0.61339558]))
('Labels:\t\t', [0.57200000762899994, 0.52700000762900001, 0.603999996185, 0.64400001525899997, 0.54449998855600013])
('Predictions:\t', array([ 0.58367911,  0.57471263,  0.60305632,  0.58382286,  0.60256298]))
('Labels:\t\t', [0.57200000762899994, 0.52700000762900001, 0.603999996185, 0.64400001525899997, 0.54449998855600013])
('Predictions:\t', array([ 0.54731101,  0.53704075,  0.64215606,  0.6447537 ,  0.60846721]))
('Labels:\t\t', [0.57200000762899994, 0.52700000762900001, 0.603999996185, 0.64400001525899997, 0.54449998855600013])
('Predictions:\t', array([ 0.53181273,  0.54129371,  0.64299529,  0.6317743 ,  0.6090176 ]))
('Labels:\t\t', [0.57200000762899994, 0.52700000762900001, 0.603999996185, 0.64400001525899997, 0.54449998855600013])
('Predictions:\t', array([ 0.58304824,  0.52895953,  0.63692701,  0.65370488,  0.61268543]))
('Labels:\t\t', [0.57200000762899994, 0.52700000762900001, 0.60

In [164]:
RMSE

{'ElasticNet': [0.035728040777230269, 6.1318727629448668],
 'Lasso': [0.035749524577990618, 6.1355599489718964],
 'LinearSVR': [0.039223400022668546, 6.7317684663630617],
 'Ridge': [0.042308115421630488, 7.2611868706430434],
 'SVR': [0.054945113174939907, 9.4300285043633405]}

on remarque aisement que les valeurs predites par differents regressons disposent d'une bonne exacitude

In [165]:
CVScore={}
for reg in medecineModel.predictive_models.keys():
    scores,Sstd,Smean=medecineModel.cross_evaluate(model=reg) #RMSE of each model
    CVScore[reg]=[Smean,Smean*100/medecineModel.dataset_bin.CGPA.mean(),Sstd]

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [166]:
CVScore

{'ElasticNet': [0.05385024858262679,
  9.2421209049912498,
  0.0077452554168906682],
 'Lasso': [0.057201142374810297, 9.8172225318598016, 0.011779836385754612],
 'LinearSVR': [0.057145769292281247, 9.8077190525428897, 0.012660484400081499],
 'Ridge': [0.058729354791166297, 10.079504030872744, 0.013786514979312933],
 'SVR': [0.055704472201285958, 9.5603545124414318, 0.0075439125128585185]}

Aussi on peut remarquer les methodes lineaires disposent des meilleurs resulats

In [167]:
RMSE={}
for reg in medecineModel.predictive_models.keys():
    rmse=medecineModel.evaluate(model=reg,sur='test') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/medecineModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.70557036,  0.52330177,  0.50976296,  0.71438698,  0.56793555]))
('Labels:\t\t', [0.60950000762899992, 0.59149999618500004, 0.5264999961850001, 0.71999999999999997, 0.625])
('Predictions:\t', array([ 0.60453002,  0.58019694,  0.57951851,  0.58866456,  0.57971231]))
('Labels:\t\t', [0.60950000762899992, 0.59149999618500004, 0.5264999961850001, 0.71999999999999997, 0.625])
('Predictions:\t', array([ 0.74322262,  0.54804358,  0.4950294 ,  0.75314708,  0.3638396 ]))
('Labels:\t\t', [0.60950000762899992, 0.59149999618500004, 0.5264999961850001, 0.71999999999999997, 0.625])
('Predictions:\t', array([ 0.74492835,  0.54652426,  0.49737902,  0.75401659,  0.36695902]))
('Labels:\t\t', [0.60950000762899992, 0.59149999618500004, 0.5264999961850001, 0.71999999999999997, 0.625])
('Predictions:\t', array([ 0.70965172,  0.52589337,  0.50557467,  0.71882489,  0.26180898]))
('Labels:\t\t', [0.60950000762899992, 0.59149999618500004, 0.5264999961850001, 0.71999999999999997, 0.6

In [168]:
RMSE

{'ElasticNet': [0.056077053166495888, 9.624299218696315],
 'Lasso': [0.081967814309116455, 14.067835712965111],
 'LinearSVR': [0.069616136983969928, 11.947962579171033],
 'Ridge': [0.069838799619457298, 11.986177351662581],
 'SVR': [0.062688859363409705, 10.759059296516314]}

In [167]:
predictedValues.columns

Index([u'ElasticNet', u'Lasso', u'LinearSVR', u'Ridge', u'SVR', u'RealValue'], dtype='object')

In [170]:
finalPred,finalRMSE=medecineModel.ensemble_methods(predictedValues)

In [171]:
finalPred.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue,finalPredict
ID,,,,,,,
11897,0.584782,0.583048,0.547311,0.531813,0.583679,0.5720,0.585433
9993,0.527989,0.528960,0.537041,0.541294,0.574713,0.5270,0.527492
10585,0.636439,0.636927,0.642156,0.642995,0.603056,0.6040,0.636381
12075,0.653175,0.653705,0.644754,0.631774,0.583823,0.6440,0.654108
10674,0.613396,0.612685,0.608467,0.609018,0.602563,0.5445,0.613186


In [172]:
finalRMSE*100/medecineModel.dataset_bin.CGPA.mean()

6.1310235543650773

On remarque que notre ensemble methode sans modele avec SVM nous donne un score de R% en valdiation croisé voyons combien il nous donnerasavec l'ensemble d'evaluation

In [174]:
predictionTest = medecineModel.predict_test()

In [175]:
predictionTest.head(1)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
10540,0.70557,0.709652,0.743223,0.744928,0.60453,0.6095


In [178]:
finalPredictTes,rmseFinalTest=medecineModel.ensemble_methods(predictedValues)

In [179]:
rmseFinalTest*100/medecineModel.y_test.mean()

6.1435529512307587

Nous avons un resulat de % sur notre ensemble d'evaluation wouhhhhh........

 10.56

esayons en fin d'anlayser les coefficients de nos modèles de regressions lineares

In [180]:
ridgeRgressorMed=medecineModel.predictive_models['Ridge']

In [181]:
sorted(zip(np.abs(ridgeRgressorMed.coef_), medecineModel.dataset_bin.columns), reverse=True)

[(0.67955374177495942, 'zanner'),
 (0.16952187391540791, 'agronomie'),
 (0.16700919859714225, 'latin philo'),
 (0.16503143409517962, 'commerciale informatique'),
 (0.15454305959150783, 'elec indust'),
 (0.1440110742725694, 'nutr'),
 (0.14105962410967732, 'secretariat'),
 (0.13297758208909966, 'sociale'),
 (0.13073182192224275, 'bio-chimie'),
 (0.12034117854254453, 'math-physique'),
 (0.070372776765852754, 'sebyera'),
 (0.065054430872411709, 'groupe scolaire gilgali'),
 (0.06287735396867368, 'commmerciale et adm'),
 (0.062877353968673666, 'kitsombiro'),
 (0.061683976163046612, 'mikeno islamique'),
 (0.061315715886909615, 'metanoia'),
 (0.060359045156389984, 'uhuru'),
 (0.056270821803897607, 'maendeleo'),
 (0.054948076892295562, 'idap isp rutshuru'),
 (0.054158058218812186, 'mont caemel'),
 (0.053458258711100638, 'lukuga'),
 (0.052874880206182792, 'kashenda'),
 (0.052617180061101443, 'la releve'),
 (0.051981186053137972, 'saint michel'),
 (0.04953018842901822, 'alfajiri'),
 (0.0487250086

In [182]:
medecine.loc[medecine.SCHOOL_RIGHT=='zanner']

,DIPPERC,SCHOOL_RIGHT,OPTION_RIGHT,FAC,CGPA,DistinctionRatio,EchecRatio,Pass1stSessionRatio
ID,,,,,,,,
10384,59.0,zanner,latin philo,FM,59.15,G,G,G
11610,52.0,zanner,latin philo,FM,49.00,G,A,G


In [48]:
predictedValues.loc[medecine.loc[medecine.SCHOOL_RIGHT=='sainte ursule'].index] + 

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
9879,0.617962,0.617356,0.625193,0.586703,0.530568,0.6155
9891,0.568561,0.568130,0.545675,0.569709,0.527591,0.5590
10250,NaN,NaN,NaN,NaN,NaN,NaN
10290,NaN,NaN,NaN,NaN,NaN,NaN
10324,0.541302,0.541910,0.535790,0.567813,0.545578,0.5525
10481,0.597378,0.596845,0.592060,0.579622,0.529327,0.5915
10535,0.601495,0.600947,0.598687,0.581038,0.529575,0.6155
10538,NaN,NaN,NaN,NaN,NaN,NaN
10550,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
medecineModel.save_models('medecine')

##### Faculté de Technologie

In [291]:
technologie=datasetCGPA.loc[datasetCGPA.FAC=='FSTA']

In [270]:
technologie.corr()

,DIPPERC,CGPA
DIPPERC,1.000000,0.353908
CGPA,0.353908,1.000000


In [271]:
technologie.shape

(903, 8)

In [292]:
technologie.head(5)

,DIPPERC,SCHOOL_RIGHT,OPTION_RIGHT,FAC,CGPA,DistinctionRatio,EchecRatio,Pass1stSessionRatio
ID,,,,,,,,
10124,50.0,zanner,commmerciale et adm,FSTA,49.250000,G,A,G
10560,51.0,zanner,latin philo,FSTA,48.750000,G,A,G
11697,52.0,zanner,commmerciale et adm,FSTA,45.099998,G,A,G
3289,61.0,edap/isp,math-physique,FSTA,49.750000,G,A,G
3691,59.0,edap/isp,commmerciale et adm,FSTA,64.300001,G,G,G


In [293]:
technologie.EchecRatio.value_counts()

G    459
A    289
D     66
E     52
C     34
F      3
Name: EchecRatio, dtype: int64

In [294]:
techModel=PredictiveModelBuilding(dataset=technologie,encoderFunction=ConvertCat)

In [213]:
type(techModel.encoders)

dict

In [273]:
techModel.scale(['DIPPERC','CGPA'])

In [295]:
trainDes,tesDes=techModel.split()

In [248]:
trainDes

,DIPPERC,CGPA
count,112.000000,112.000000
mean,0.537500,0.618471
std,0.039421,0.062725
min,0.500000,0.410000
25%,0.510000,0.591500
50%,0.520000,0.620500
75%,0.552500,0.660000
max,0.650000,0.732500


In [249]:
tesDes

,DIPPERC,CGPA
count,28.000000,28.000000
mean,0.540714,0.626101
std,0.043879,0.061257
min,0.500000,0.430000
25%,0.510000,0.606500
50%,0.520000,0.633500
75%,0.570000,0.654250
max,0.690000,0.734000


In [296]:
predictedValues=techModel.train()

/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[LibSVM]

In [297]:
predictedValues.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
10201,60.604003,60.607533,62.391937,63.058521,57.198339,57.350000
11607,63.009612,63.012679,66.544251,65.798714,61.877645,68.300003
9526,53.268875,53.266158,49.263117,51.279059,53.127159,50.299999
9263,52.235284,52.230924,53.460321,51.116012,54.507480,49.250000
11484,70.644173,70.654126,82.198513,79.615717,63.691280,76.000000


In [298]:
RMSE={}
for reg in techModel.predictive_models.keys():
    rmse=techModel.evaluate(model=reg,sur='train') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/techModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 60.60400319,  63.00961245,  53.26887467,  52.23528392,  70.64417258]))
('Labels:\t\t', [57.350000381499996, 68.300003051800005, 50.299999237100003, 49.25, 76.0])
('Predictions:\t', array([ 57.19833938,  61.87764473,  53.12715897,  54.5074802 ,  63.69127954]))
('Labels:\t\t', [57.350000381499996, 68.300003051800005, 50.299999237100003, 49.25, 76.0])
('Predictions:\t', array([ 62.39193712,  66.54425107,  49.26311735,  53.46032132,  82.19851316]))
('Labels:\t\t', [57.350000381499996, 68.300003051800005, 50.299999237100003, 49.25, 76.0])
('Predictions:\t', array([ 63.05852147,  65.79871437,  51.27905858,  51.11601226,  79.61571675]))
('Labels:\t\t', [57.350000381499996, 68.300003051800005, 50.299999237100003, 49.25, 76.0])
('Predictions:\t', array([ 60.60753335,  63.01267949,  53.26615819,  52.23092374,  70.65412638]))
('Labels:\t\t', [57.350000381499996, 68.300003051800005, 50.299999237100003, 49.25, 76.0])


In [194]:
techModel.dataset_bin.shape

(903, 243)

In [299]:
RMSE

{'ElasticNet': [5.3690359923129964, 9.5077858966943065],
 'Lasso': [5.369024231688817, 9.5077650703305299],
 'LinearSVR': [6.936412022623557, 12.283382062773185],
 'Ridge': [5.9145648893871803, 10.473838640849342],
 'SVR': [7.0841334430944523, 12.544975324618273]}

on remarque aisement que les valeurs predites par differents regressons disposent d'une bonne exacitude

In [300]:
CVScore={}
for reg in techModel.predictive_models.keys():
    scores,Sstd,Smean=techModel.cross_evaluate(model=reg) #RMSE of each model
    CVScore[reg]=[Smean,Smean*100/techModel.dataset_bin.CGPA.mean(),Sstd]

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [301]:
CVScore

{'ElasticNet': [6.9506390515230336, 12.30857607244568, 0.46049815937403449],
 'Lasso': [7.3522246143385832, 13.019725998786338, 0.86701681117023466],
 'LinearSVR': [7.2591556537260402, 12.854914335687257, 0.48738022242329471],
 'Ridge': [7.1499702003302872, 12.661562695764072, 0.44050073824004099],
 'SVR': [7.1092185822378244, 12.589397476472989, 0.53038032407321345]}

Aussi on peut remarquer les methodes lineaires disposent des meilleurs resulats

In [302]:
RMSE={}
for reg in techModel.predictive_models.keys():
    rmse=techModel.evaluate(model=reg,sur='test') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/techModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 62.20267384,  57.32076694,  52.94094449,  61.52279276,  59.73717494]))
('Labels:\t\t', [61.300001144399992, 61.366668701199998, 42.0, 53.024999618500004, 62.100000381499996])
('Predictions:\t', array([ 60.45591946,  53.71380708,  53.01518037,  57.22771924,  60.56086696]))
('Labels:\t\t', [61.300001144399992, 61.366668701199998, 42.0, 53.024999618500004, 62.100000381499996])
('Predictions:\t', array([ 65.93307692,  53.50806502,  49.43720392,  59.9503175 ,  63.64780075]))
('Labels:\t\t', [61.300001144399992, 61.366668701199998, 42.0, 53.024999618500004, 62.100000381499996])
('Predictions:\t', array([ 64.75950753,  55.80947   ,  50.89830375,  60.67223756,  63.60471499]))
('Labels:\t\t', [61.300001144399992, 61.366668701199998, 42.0, 53.024999618500004, 62.100000381499996])
('Predictions:\t', array([ 62.20569988,  57.3200994 ,  52.94338536,  61.52546259,  59.7305908 ]))
('Labels:\t\t', [61.300001144399992, 61.366668701199998, 42.0, 53.024999618500004, 62.10000038

In [303]:
RMSE

{'ElasticNet': [6.8911085821996876, 12.203156224736247],
 'Lasso': [7.1968248545011617, 12.744535508901874],
 'LinearSVR': [6.8505261279869316, 12.131290570199806],
 'Ridge': [6.8479936332655109, 12.12680588847477],
 'SVR': [6.825255335717169, 12.086539653520397]}

In [58]:
predictedValues.columns

Index([u'ElasticNet', u'Lasso', u'LinearSVR', u'Ridge', u'SVR', u'RealValue'], dtype='object')

In [311]:
finalPred,finalRMSE=medecineModel.ensemble_methods(predictedValues)

In [312]:
finalPred.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue,finalPredict
ID,,,,,,,
10201,60.604003,60.607533,62.391937,63.058521,57.198339,57.350000,60.450195
11607,63.009612,63.012679,66.544251,65.798714,61.877645,68.300003,63.200195
9526,53.268875,53.266158,49.263117,51.279059,53.127159,50.299999,53.372070
9263,52.235284,52.230924,53.460321,51.116012,54.507480,49.250000,52.168945
11484,70.644173,70.654126,82.198513,79.615717,63.691280,76.000000,70.512695


In [305]:
finalRMSE*100/techModel.dataset_bin.CGPA.mean()

9.505133680383377

On remarque que notre ensemble methode sans modele avec SVM nous donne un score de R% en valdiation croisé voyons combien il nous donnerasavec l'ensemble d'evaluation

In [306]:
predictionTest=techModel.predict_test()

In [169]:
predictionTest.head(1)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
10825,0.551732,0.552633,0.55165,0.560239,0.563527,0.543


In [308]:
predictionTest.columns

Index([u'ElasticNet', u'Lasso', u'LinearSVR', u'Ridge', u'SVR', u'RealValue'], dtype='object')

In [307]:
finalPredictTes,rmseFinalTest=techModel.ensemble_methods(predictionTest)

ValueError: labels ['finalPredict'] not contained in axis

In [206]:
rmseFinalTest*100/techModel.y_test.mean()

11.005597895855978

Nous avons un resulat de % sur notre ensemble d'evaluation wouhhhhh........

In [309]:
techModel.save_models('technologie')

In [318]:
sorted(['medecine.pkl','technologie.pkl','droit.pkl','sante.pkl','theologie.pkl'],reverse=True)

['theologie.pkl', 'technologie.pkl', 'sante.pkl', 'medecine.pkl', 'droit.pkl']

In [319]:
medecineModel.save_models('medecine')

PicklingError: Can't pickle <class '__main__.PredictiveModelBuilding'>: it's not the same object as __main__.PredictiveModelBuilding

#### Faculté d'économie

In [360]:
economie = datasetCGPA.loc[datasetCGPA.FAC=='FSEG']

In [323]:
economie.head(5)

,DIPPERC,SCHOOL_RIGHT,OPTION_RIGHT,FAC,CGPA,DistinctionRatio,EchecRatio,Pass1stSessionRatio
ID,,,,,,,,
3895,52.0,zanner,commmerciale et adm,FSEG,59.400002,G,G,G
4048,53.0,zanner,commmerciale et adm,FSEG,49.250000,G,A,G
4217,54.0,zanner,commmerciale et adm,FSEG,61.299999,G,G,A
4347,53.0,zanner,commmerciale et adm,FSEG,60.599998,G,G,G
4409,58.0,zanner,commmerciale et adm,FSEG,48.599998,G,A,G


In [352]:
economieModel = predictiveModelBuilding.PredictiveModelBuilding(dataset=economie,encoderFunction=ConvertCat)

NameError: name 'predictiveModelBuilding' is not defined

In [325]:
economieModel.scale(['DIPPERC','CGPA'])

In [326]:
trainDes,tesDes=economieModel.split()

In [248]:
trainDes

,DIPPERC,CGPA
count,112.000000,112.000000
mean,0.537500,0.618471
std,0.039421,0.062725
min,0.500000,0.410000
25%,0.510000,0.591500
50%,0.520000,0.620500
75%,0.552500,0.660000
max,0.650000,0.732500


In [249]:
tesDes

,DIPPERC,CGPA
count,28.000000,28.000000
mean,0.540714,0.626101
std,0.043879,0.061257
min,0.500000,0.430000
25%,0.510000,0.606500
50%,0.520000,0.633500
75%,0.570000,0.654250
max,0.690000,0.734000


In [327]:
predictedValues=economieModel.train()

[LibSVM]

In [328]:
predictedValues.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
10522,0.496530,0.502412,0.499731,0.504135,0.553606,0.490000
10063,0.588449,0.586442,0.576245,0.573654,0.582586,0.582500
10177,0.534957,0.535226,0.533751,0.533738,0.552965,0.546000
9896,0.596477,0.597554,0.603830,0.605007,0.565531,0.631500
8240,0.597547,0.596419,0.591461,0.590058,0.574581,0.581333


In [329]:
RMSE={}
for reg in economieModel.predictive_models.keys():
    rmse=economieModel.evaluate(model=reg,sur='train') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/economieModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.49653044,  0.58844851,  0.53495734,  0.59647727,  0.59754661]))
('Labels:\t\t', [0.48999999999999999, 0.58250000000000002, 0.54600000381499991, 0.63149999618500008, 0.58133333841899992])
('Predictions:\t', array([ 0.55360566,  0.58258566,  0.55296459,  0.56553092,  0.57458096]))
('Labels:\t\t', [0.48999999999999999, 0.58250000000000002, 0.54600000381499991, 0.63149999618500008, 0.58133333841899992])
('Predictions:\t', array([ 0.49973114,  0.57624453,  0.53375134,  0.60383049,  0.59146069]))
('Labels:\t\t', [0.48999999999999999, 0.58250000000000002, 0.54600000381499991, 0.63149999618500008, 0.58133333841899992])
('Predictions:\t', array([ 0.50413497,  0.57365408,  0.53373835,  0.6050066 ,  0.59005813]))
('Labels:\t\t', [0.48999999999999999, 0.58250000000000002, 0.54600000381499991, 0.63149999618500008, 0.58133333841899992])
('Predictions:\t', array([ 0.50241228,  0.58644249,  0.5352264 ,  0.59755388,  0.59641852]))
('Labels:\t\t', [0.48999999999999999, 0.582

In [330]:
RMSE

{'ElasticNet': [0.040280083975281308, 7.0679999185158024],
 'Lasso': [0.040481939789675847, 7.1034198267902751],
 'LinearSVR': [0.04302405543393948, 7.5494882405885013],
 'Ridge': [0.04502992329990739, 7.9014605433750331],
 'SVR': [0.050355701637281383, 8.835981952955331]}

on remarque aisement que les valeurs predites par differents regressons disposent d'une bonne exacitude

In [331]:
CVScore={}
for reg in economieModel.predictive_models.keys():
    scores,Sstd,Smean=economieModel.cross_evaluate(model=reg) #RMSE of each model
    CVScore[reg]=[Smean,Smean*100/economieModel.dataset_bin.CGPA.mean(),Sstd]

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [332]:
CVScore

{'ElasticNet': [0.049537290152297062,
  8.6923742009771718,
  0.003960725117337271],
 'Lasso': [0.056998605089412901, 10.001621058553694, 0.012626347994924018],
 'LinearSVR': [0.055175691351229718,
  9.6817519599478707,
  0.0071690146180578094],
 'Ridge': [0.055265117259548649, 9.6974436430411668, 0.0060450687825310406],
 'SVR': [0.050842564690547223, 8.9214124605710303, 0.0038018927432977869]}

Aussi on peut remarquer les methodes lineaires disposent des meilleurs resulats

In [333]:
RMSE={}
for reg in economieModel.predictive_models.keys():
    rmse=economieModel.evaluate(model=reg,sur='test') #RMSE of each model
    RMSE[reg]=[rmse,rmse*100/economieModel.dataset_bin.CGPA.mean()]

('Predictions:\t', array([ 0.58544765,  0.63471039,  0.55896242,  0.57051863,  0.61680775]))
('Labels:\t\t', [0.48475000381499994, 0.637000007629, 0.618333333333, 0.56499999999999995, 0.63449997901900002])
('Predictions:\t', array([ 0.57708787,  0.58486241,  0.56704903,  0.5668039 ,  0.58381236]))
('Labels:\t\t', [0.48475000381499994, 0.637000007629, 0.618333333333, 0.56499999999999995, 0.63449997901900002])
('Predictions:\t', array([ 0.5988534 ,  0.65187643,  0.52713843,  0.54758083,  0.63116848]))
('Labels:\t\t', [0.48475000381499994, 0.637000007629, 0.618333333333, 0.56499999999999995, 0.63449997901900002])
('Predictions:\t', array([ 0.60155102,  0.65533481,  0.52761055,  0.53789522,  0.63482564]))
('Labels:\t\t', [0.48475000381499994, 0.637000007629, 0.618333333333, 0.56499999999999995, 0.63449997901900002])
('Predictions:\t', array([ 0.58718345,  0.63770547,  0.5532695 ,  0.56222642,  0.61925501]))
('Labels:\t\t', [0.48475000381499994, 0.637000007629, 0.618333333333, 0.56499999999

In [334]:
RMSE

{'ElasticNet': [0.048621933788682618, 8.5317554021829558],
 'Lasso': [0.05811043125161957, 10.196714674264697],
 'LinearSVR': [0.054365502583516578, 9.5395870591077188],
 'Ridge': [0.054620968718741239, 9.584414041694437],
 'SVR': [0.049902791220194295, 8.7565091595760762]}

In [167]:
predictedValues.columns

Index([u'ElasticNet', u'Lasso', u'LinearSVR', u'Ridge', u'SVR', u'RealValue'], dtype='object')

In [335]:
finalPred,finalRMSE=economieModel.ensemble_methods(predictedValues)

In [336]:
finalPred.head(5)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue,finalPredict
ID,,,,,,,
10522,0.496530,0.502412,0.499731,0.504135,0.553606,0.490000,0.491606
10063,0.588449,0.586442,0.576245,0.573654,0.582586,0.582500,0.587983
10177,0.534957,0.535226,0.533751,0.533738,0.552965,0.546000,0.534224
9896,0.596477,0.597554,0.603830,0.605007,0.565531,0.631500,0.597209
8240,0.597547,0.596419,0.591461,0.590058,0.574581,0.581333,0.597677


In [337]:
finalRMSE*100/economieModel.dataset_bin.CGPA.mean()

7.0596953935752484

On remarque que notre ensemble methode sans modele avec SVM nous donne un score de R% en valdiation croisé voyons combien il nous donnerasavec l'ensemble d'evaluation

In [342]:
predictionTest=economieModel.predict_test()

In [343]:
predictionTest.head(1)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue
ID,,,,,,
8600,0.585448,0.587183,0.598853,0.601551,0.577088,0.48475


In [344]:
finalPredictTes,rmseFinalTest=economieModel.ensemble_methods(predictionTest)

In [345]:
rmseFinalTest*100/economieModel.y_test.mean()

8.2260892879413188

In [346]:
economieModel.predict_new(newStData)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,finalOutput
index,,,,,,
0,0.599913,0.226197,0.375919,0.402569,0.565043,0.588466


In [347]:
economieModel.save_models('economie')

In [348]:
PredictiveModelBuilding.__module__='predictiveModelBuilding'

In [351]:
economieModel.save_models('econommie')

PicklingError: Can't pickle <class 'predictiveModelBuilding.PredictiveModelBuilding'>: it's not found as predictiveModelBuilding.PredictiveModelBuilding

#### Analyse des resultats

In [232]:
ridgeRgressorTech=techModel.predictive_models['Ridge']

voyons dans notre modèle quel sont les attribues le plus importans

In [233]:
max(ridgeRgressorTech.coef_)

0.68950896599836575

In [234]:
ridgeRgressorTech.intercept_

0.0

In [236]:
sorted(zip(ridgeRgressorTech.coef_[:23], techModel.x_train.columns[:23]),reverse=True)

[(0.19071441376543907, 'elec indust'),
 (0.1649118473838623, 'math-physique'),
 (0.16256870207068552, 'elec'),
 (0.14734984792880831, 'mec gene'),
 (0.14723708779257388, 'bio-chimie'),
 (0.11622973859108161, 'commerciale informatique'),
 (0.11617773369178477, 'inconnu'),
 (0.11517421220594158, 'electronique g\xc3\xa9n\xc3\xa9rale'),
 (0.11239991574264298, 'pedagogie'),
 (0.11091223188143225, 'commmerciale et adm'),
 (0.10875688450666228, 'construction'),
 (0.10589514494204152, 'sociale'),
 (0.10129123395852929, 'latin philo'),
 (0.090579627882037897, 'machine outil'),
 (0.080157532104476362, 'mecanique machines outils'),
 (0.077790193679636174, 'agrecole'),
 (0.066887253301075242, 'nutr'),
 (0.06419493324031593, 'industrielle'),
 (0.050676910236558362, 'batiment'),
 (0.050469853123232045, 'agronomie'),
 (0.036869783807009844, 'secretariat'),
 (0.0, 'imprimerie'),
 (0.0, "hotesse d'acceuil")]

Nous remrqouns dans ces resulat que concernant les valeurs de coefficient de notre modèle de regression les options avec un bon score sont les options batiment , elec insustrielle , elec math physique et les option avec un moindre score en technologie soont le soptions imprimerie , latin philo,pedagogie,..

Essayons de verifier les écoles :

In [224]:
sorted(zip(ridgeRgressorTech.coef_[24:236], techModel.X_train.columns[24:236]),reverse=True)[:23]

[(0.12221946644343301, 'itfm/bukavu'),
 (0.11544826021849611, 'vungi'),
 (0.10601692142687072, 'nikisi'),
 (0.10135463091815416, 'Lwanga'),
 (0.093502748892734133, 'it bugabo'),
 (0.090555922000722253, 'cirezi'),
 (0.088121403276466337, 'itk mahamba'),
 (0.079465114538439616, 'katana'),
 (0.079151243770063037, 'technique ind. de mahamba'),
 (0.07774206929450149, 'du lac'),
 (0.076335377719177949, 'mgr guido'),
 (0.075944583258854809, 'avenir'),
 (0.075030717919957607, 'saint michel'),
 (0.072977785314089133, 'de beni'),
 (0.066975528149117669, 'action kusaidiya'),
 (0.061269110455925049, 'namurera'),
 (0.060447588998714233, 'it salama'),
 (0.058463266084065074, 'kambali'),
 (0.058443609291880839, 'itig'),
 (0.057776867646034659, 'iti gombe'),
 (0.054440832383465088, 'uenezaji'),
 (0.052474455779097019, 'humule'),
 (0.05038194941607773, 'malikia wa bingu')]

In [241]:
print (ridgeRgressorTech.coef_[239], techModel.x_train.columns[239])

(0.68950896599836575, 'DIPPERC')


In [228]:
technologie.loc[operator.and_(technologie.SCHOOL_RIGHT=='itfm/bukavu',technologie.CGPA>68)]

,DIPPERC,SCHOOL_RIGHT,OPTION_RIGHT,FAC,CGPA,DistinctionRatio,EchecRatio,Pass1stSessionRatio
ID,,,,,,,,
2457,56.0,itfm/bukavu,elec indust,FSTA,76.400002,A,G,A
4071,63.0,itfm/bukavu,machine outil,FSTA,71.199997,A,G,G
4282,54.0,itfm/bukavu,elec indust,FSTA,70.750000,D,G,G
4313,51.0,itfm/bukavu,elec,FSTA,68.600002,D,G,G
9958,59.0,itfm/bukavu,mecanique machines outils,FSTA,71.699997,D,G,A
10043,58.0,itfm/bukavu,elec indust,FSTA,79.099998,A,G,A
10051,60.0,itfm/bukavu,elec indust,FSTA,72.500000,A,G,D
10142,56.0,itfm/bukavu,elec indust,FSTA,69.650002,D,G,A
10199,65.0,itfm/bukavu,mecanique machines outils,FSTA,83.549999,A,G,D


In [266]:
predictedValues.head(1)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue,finalPredict
ID,,,,,,,
10522,0.49653,0.502412,0.499731,0.504135,0.553606,0.49,0.491606


In [229]:
pred,error=techModel.ensembelMethods(predictedValues.loc[technologie.loc[operator.and_(technologie.SCHOOL_RIGHT=='itig',technologie.CGPA>65)].index].dropna())

In [230]:
pred

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue,finalPredict
ID,,,,,,,
3627,0.574900,0.572014,0.558456,0.555100,0.569956,0.670000,0.646194
4074,0.605507,0.605690,0.606418,0.605682,0.580733,0.654000,0.662018
4106,0.574900,0.572014,0.558456,0.555100,0.569956,0.672000,0.646194
7644,0.631069,0.635201,0.652961,0.656045,0.575412,0.739500,0.719650
8460,0.663395,0.666978,0.681573,0.684457,0.611192,0.691000,0.695236
8663,0.627324,0.630988,0.646661,0.649316,0.575049,0.723000,0.701324
9036,0.601763,0.601477,0.600118,0.598952,0.580368,0.719333,0.665863
9078,0.639209,0.643602,0.663121,0.666249,0.584008,0.807667,0.745544
9295,0.648416,0.650128,0.656371,0.657538,0.609739,0.678000,0.688095


##### Traitement des nouvelles valeurs

Dans cette partie nous allons essayer de predire la valeur du CGPA pour des nouvelles variables qui ne sont pas dans notre ensemble d'apprentissage'

In [86]:
technologie.columns

Index([u'DIPPERC', u'SCHOOL_RIGHT', u'OPTION_RIGHT', u'FAC', u'CGPA',
       u'DistinctionRatio', u'EchecRatio', u'Pass1stSessionRatio'],
      dtype='object')

In [246]:
newStudent={'DIPPERC':0.60,'SCHOOL_RIGHT':'itfm/bukavu','OPTION_RIGHT':'elec indust'}

In [247]:
newStData=pd.DataFrame(newStudent,columns=newStudent.keys(),index=range(1))

In [248]:
newStData

,DIPPERC,OPTION_RIGHT,SCHOOL_RIGHT
0,0.6,elec indust,itfm/bukavu


In [231]:
newStData[['OPTION_RIGHT','SCHOOL_RIGHT']]

,OPTION_RIGHT,SCHOOL_RIGHT
0,elec indust,itfm/bukavu


In [64]:
optionEnc=techModel.encoders['OPTION_RIGHT']

In [65]:
schoolEnc=techModel.encoders['SCHOOL_RIGHT']

In [157]:
Options=pd.DataFrame(data=dict(zip(optionEnc.classes_,optionEnc.transform(newStData[['OPTION_RIGHT']])[0])),
             index=newStData.index, columns=optionEnc.classes_)

In [158]:
Schools=pd.DataFrame(data=dict(zip(schoolEnc.classes_,schoolEnc.transform(newStData[['SCHOOL_RIGHT']])[0])),
             index=newStData.index, columns=schoolEnc.classes_)

In [94]:
Schools

,54,61,Bungulu Beni,INST DE KATWA,Institut MWANDA,LWANGA,Lwanga,MWANDU,Mwanda,action kusaidiya,...,uhuru,uzima,visoke,vungi,wai wai,wapole,weza,wima,zanner,zawadi ya raisi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
newStData['DIPPERC']

0    0.60
Name: DIPPERC, dtype: object

In [159]:
Schools.reset_index(inplace=True)
Options.reset_index(inplace=True)

In [97]:
techModel.X_train.shape

(722, 237)

In [162]:
New_X=pd.merge(Options,Schools,on='index')

In [163]:
New_X['DIPPERC']=newStData['DIPPERC']

In [164]:
New_X.set_index(keys=['index'],inplace=True)

In [113]:
New_X.shape

(1, 237)

In [109]:
len(ridgeRgressorTech.coef_)

237

In [165]:
ridgeRgressorTech.predict(New_X)

array([ 0.68790015])

In [160]:
techModel.X_train.head(1)

,agrecole,agronomie,batiment,bio-chimie,commerciale informatique,commmerciale et adm,construction,elec,elec indust,electronique générale,...,uzima,visoke,vungi,wai wai,wapole,weza,wima,zanner,zawadi ya raisi,DIPPERC
ID,,,,,,,,,,,,,,,,,,,,,
9486,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.74


In [161]:
New_X.head(1)

,agrecole,agronomie,batiment,bio-chimie,commerciale informatique,commmerciale et adm,construction,elec,elec indust,electronique générale,...,uzima,visoke,vungi,wai wai,wapole,weza,wima,zanner,zawadi ya raisi,DIPPERC
index,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.60


Essayons de prédire la nouvelles valeurs avec la nouvelle methode

In [260]:
new_student_data = newStData
option_encoder = techModel.encoders['OPTION_RIGHT']
school_encoder = techModel.encoders['SCHOOL_RIGHT']
options = pd.DataFrame(data=dict(zip(option_encoder.classes_, option_encoder.transform(new_student_data[['OPTION_RIGHT']])[0])),index=new_student_data.index, columns=option_encoder.classes_)
schools = pd.DataFrame(data=dict(zip(school_encoder.classes_, school_encoder.transform(new_student_data[['SCHOOL_RIGHT']])[0])),index=new_student_data.index, columns=school_encoder.classes_)
schools.reset_index(inplace=True)
options.reset_index(inplace=True)
new_dataset = pd.merge(options, schools, on='index')
new_dataset['DIPPERC'] = new_student_data['DIPPERC']
new_dataset.set_index(keys=['index'], inplace=True)
predictions = {}
for clf in techModel.predictive_models.values():
    predictions[clf.__class__.__name__] = clf.predict(new_dataset)
predicted_values = pd.DataFrame.from_dict(predictions, dtype=np.float)
predicted_values.set_index(new_dataset.index, inplace=True)
    #predicted_values.loc[:, 'finalOutput'] = techModel.stacker.predict(predicted_values)

In [265]:
techModel.stacker.intercept_

-352031538999.05536

In [261]:
predicted_values

,ElasticNet,Lasso,LinearSVR,Ridge,SVR
index,,,,,
0,0.698827,0.700242,0.704728,0.703815,0.624719


In [257]:
def predictNew(self,newStData):
    """this call will handle predictions for new values,but frirst it will endcode them nand then try to predict"""
    #start first by handling categorical values
    optionEnc=self.encoders['OPTION_RIGHT']
    schoolEnc=self.encoders['SCHOOL_RIGHT']
    Options=pd.DataFrame(data=dict(zip(optionEnc.classes_,optionEnc.transform(newStData[['OPTION_RIGHT']])[0])),
             index=newStData.index, columns=optionEnc.classes_)
    Schools=pd.DataFrame(data=dict(zip(schoolEnc.classes_,schoolEnc.transform(newStData[['SCHOOL_RIGHT']])[0])),
             index=newStData.index, columns=schoolEnc.classes_)
    Schools.reset_index(inplace=True)
    Options.reset_index(inplace=True)
    X=pd.merge(Options,Schools,on='index')
    X['DIPPERC']=newStData['DIPPERC']
    X.set_index(keys=['index'],inplace=True)
    predictions={}
    for clf in self.predictiveModels.values():
      predictions[clf.__class__.__name__]= clf.predict(X)
    predictedVal=pd.DataFrame.from_dict(predictions,dtype=np.float)
    predictedVal.set_index(X.index,inplace=True)
    predictedVal.loc[:,'finalOutput']=self.stacker.predict(predictedVal)
    return predictedVal

In [258]:
predictedVal

NameError: name 'predictedVal' is not defined

In [240]:
predictNew(self=techModel,newStData=newStData)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,finalOutput
index,,,,,,
0,0.696491,0.698198,0.702101,0.701125,0.628597,0.680711


In [259]:
newStudent={'DIPPERC':0.7,'SCHOOL_RIGHT':'itfm/bukavu','OPTION_RIGHT':'elec'}

In [260]:
newStData2=pd.DataFrame(newStudent,columns=newStudent.keys(),index=range(1))

In [261]:
predictNew(self=techModel,newStData=newStData2)

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,finalOutput
index,,,,,,
0,0.702262,0.708521,0.734052,0.736755,0.603966,0.684074


essayons en faculté de Médecine

In [274]:
LassoMed=medecineModel.predictiveModels['Lasso']

In [280]:
len(medecine.OPTION_RIGHT.value_counts())

13

In [281]:
sorted(zip(LassoMed.coef_[:13], medecineModel.X_train.columns[:13]),reverse=True)

[(0.35757368827182023, 'commerciale informatique'),
 (0.34979805279513343, 'commmerciale et adm'),
 (0.34775736681271191, 'math-physique'),
 (0.33985943323756168, 'latin philo'),
 (0.33015491684910009, 'bio-chimie'),
 (0.32823217678920358, 'v\xc3\xa9t\xc3\xa9rinaire'),
 (0.29493952634047493, 'sociale'),
 (0.28920522583779829, 'pedagogie'),
 (0.2819322147819795, 'elec indust'),
 (0.23277919234663122, 'nutr'),
 (0.21194946979189991, 'agrecole'),
 (0.0, 'secretariat'),
 (0.0, 'agronomie')]

Essayons de verifier les écoles :

In [288]:
len(medecine.SCHOOL_RIGHT.value_counts()) #nombre des colones

94

In [291]:
medecineModel.X_train.columns[12]

'v\xc3\xa9t\xc3\xa9rinaire'

In [287]:
sorted(zip(LassoMed.coef_[13:13+94], medecineModel.X_train.columns[13:13+94]),reverse=True)[:23]

[(-0.1441318839144477, 'masisi'),
 (-0.13962352198707761, 'majengo'),
 (-0.12141658508131777, 'gs kigali'),
 (-0.11251800352776459, 'hekima'),
 (-0.10720237938597295, 'ngoma'),
 (-0.095335844000001099, 'de r\xc3\xa9cup\xc3\xa9ration de la gombe'),
 (-0.094776634847224872, 'amen'),
 (-0.089211977017360722, 'neema kwetu'),
 (-0.089042334344549348, 'de bukavu'),
 (-0.086953345697900128, 'Mwangaza'),
 (-0.071324022409169338, 'lukuga'),
 (-0.069783695396088016, 'kalungu'),
 (-0.069305925587554135, 'zanner'),
 (-0.06744266888518366, 'bimenya'),
 (-0.052480726765823014, 'communautaire du lac'),
 (-0.050089729535492775, 'ndosho'),
 (-0.047913018684819217, 'Intitut ALLELUYA'),
 (-0.034522842170380694, 'bethanie'),
 (-0.033870787425376019, 'himbi'),
 (-0.03241010488706185, 'la fontaine'),
 (-0.030346701263338913, 'amani'),
 (-0.028094236685508724, 'matanda'),
 (-0.021924696440913535, 'Lwanga'),
 (-0.020798051866254483, 'icl'),
 (-0.019862713511119887, 'milima'),
 (-0.018174782561455084, 'mont ca

In [296]:
medecineModel.X_train.shape

(193, 108)

In [298]:
print (LassoMed.coef_[107], medecineModel.X_train.columns[107])

(0.44356521067840698, 'DIPPERC')


In [303]:
medecine.loc[operator.and_(medecine.SCHOOL_RIGHT=='amani',True)]

,DIPPERC,SCHOOL_RIGHT,OPTION_RIGHT,FAC,CGPA,DistinctionRatio,EchecRatio,Pass1stSessionRatio
ID,,,,,,,,
10116,77.0,amani,bio-chimie,FM,62.950001,G,G,G
10118,72.0,amani,bio-chimie,FM,66.500000,G,G,A
10208,76.0,amani,bio-chimie,FM,58.349998,G,G,G
10289,73.0,amani,bio-chimie,FM,40.000000,G,A,G
10377,74.0,amani,bio-chimie,FM,58.949999,G,G,D
10437,75.0,amani,bio-chimie,FM,60.750000,G,G,A
11269,81.0,amani,bio-chimie,FM,74.000000,A,G,A
11781,64.0,amani,bio-chimie,FM,61.000000,G,G,A


In [304]:
medecineModel.ensembelMethods()

TypeError: drop() got an unexpected keyword argument 'axis'

In [305]:
pred,error=techModel.ensembelMethods(predictedValues.loc[medecine.loc[operator.and_(medecine.SCHOOL_RIGHT=='amani',True)].index].dropna())

In [306]:
pred

,ElasticNet,Lasso,LinearSVR,Ridge,SVR,RealValue,finalPredict
ID,,,,,,,
10116,0.640802,0.641353,0.647128,0.649566,0.587390,0.6295,0.62500
10118,0.620218,0.619175,0.613996,0.616084,0.586137,0.6650,0.71875
10208,0.636685,0.636918,0.640501,0.642870,0.587140,0.5835,0.56250
10377,0.628452,0.628046,0.627248,0.629477,0.586639,0.5895,0.59375
10437,0.632568,0.632482,0.633875,0.636173,0.586890,0.6075,0.62500
11269,0.657269,0.659096,0.673634,0.676351,0.588391,0.7400,0.78125


In [307]:
error

0.029933779762049806

In [308]:
predictNew(self=medecineModel,newStData=newStData2)

ValueError: shapes (1,5) and (3,) not aligned: 5 (dim 1) != 3 (dim 0)

In [68]:
from predictiveModelBuilding import PredictiveModelBuilding

In [69]:
def convert_cat(dataset, cat_col, num_col):
    """

    this function will binarize a dataset given in parametrer and
    return the dataset with categorical columns binarise by one-hot
    encoding

    """
    encs = {}
    x_train_1 = dataset[cat_col]
    x_new = dataset[num_col]
    cat_col = x_train_1.columns
    for col in cat_col:
        data = dataset[[col]]
        enc = LabelBinarizer()
        enc.fit(data)
        # Fitting One Hot Encoding on train data
        temp = enc.transform(dataset[[col]])
        # Changing the encoded features into a data frame with new column names
        temp = pd.DataFrame(temp, columns=enc.classes_)
        # In side by side concatenation index values should be same
        # Setting the index values similar to the X_train data frame
        temp = temp.set_index(dataset.index)
        # adding the new One Hot Encoded varibales to the train data frame

        x_new = pd.merge(temp, x_new, right_index=True, left_index=True)
        #saving the encoder into a dict for others operations
        encs[col] = enc
    return x_new, encs

In [70]:
def final_job(dataframe):
    """"

    this method will handle all the task related to training models in
    each departement and in final they will return a dataset with

    """

    #first we iterate over the whole dataset to get each departement

    departement_names = ['droit', 'medecine', 'psycologie', 'sante', 'economie','techonologie', 'theologie']
    results = {}
    for departement, datas in dataframe.groupby('FAC'):
        results[departement] = []
        predictive_model = PredictiveModelBuilding(dataset=datas, encoderFunction=convert_cat)
        predictive_model.scale(['DIPPERC', 'CGPA'])
        train_des, test_des = predictive_model.split()
        results[departement].append(predictive_model.dataset_bin.shape)
        print train_des
        print test_des
        predicted_values = predictive_model.train() #trainig the models
        rmse = {}
        for name, model in predictive_model.predictive_models:
            cgpa_mean = predictive_model.dataset_bin.CGPA.mean()
            rmse = predictive_model.evaluate(model=model, sur='train') #rmse of each model
            rmse[name] = [rmse, rmse*100/cgpa_mean]
            scores, score_std, score_mean = predictive_model.cross_evaluate(model=model)
            cv_score = [score_mean, score_mean*100/cgpa_mean, score_std]
            rmse[name].append(cv_score)
            print (model, scores)
            rmse_test = predictive_model.evaluate(model=model, sur='test') #rmse of each model
            rmse[name].append([rmse_test, rmse_test*100/cgpa_mean])
        final_predict, final_rmse = predictive_model.ensemble_methods(predicted_values)
        print final_predict.head(5)
        new_student = {'DIPPERC':0.60, ' SCHOOL_RIGHT':'itfm/bukavu', 'OPTION_RIGHT':'elec indust'}
        new_student_data = pd.DataFrame(new_student, columns=new_student.keys(), index=range(1))
        print predictive_model.predict_new(new_student_data)
        results[departement].append(rmse)
        results[departement].append(final_rmse)
        predictive_model.save_models(departement)
    return results


In [72]:
final_job(datasetCGPA)

TypeError: convert_cat() got an unexpected keyword argument 'catCol'